# Elliptic Data Set


## Objective

Find fraud detection.

### This Project

This project applies graph-theoretic and topological methods to Bitcoin transaction data.

## Content

Here is the three data sets using Weber, Domeniconi, Chen, Weidele, Bellei, Robinson, and Leiserson, *Anti-money laundering in Bitcoin: Experimenting with graph convolutional networks for financial forensics* [9] and it worth repeating again.

### `classes`

This anonymized data set is a transaction graph collected from the Bitcoin blockchain. A node in the graph represents a transaction, an edge can be viewed as a flow of Bitcoins between one transaction and the other. Each node has 166 features and has been labeled as being created by a "licit", "illicit" or "unknown" entity.

## Nodes and edges

### `edgelist`

The graph is made of 203,769 nodes and 234,355 edges. Two percent (4,545) of the nodes are labelled class1 (illicit). Twenty-one percent (42,019) are labelled class2 (licit). The remaining transactions are not labelled with regard to licit versus illicit.

## Features

### `features`

There are 166 features associated with each node. Due to intellectual property issues, we cannot provide an exact description of all the features in the dataset. There is a time step associated to each node, representing a measure of the time when a transaction was broadcasted to the Bitcoin network. The time steps, running from 1 to 49, are evenly spaced with an interval of about two weeks. Each time step contains a single connected component of transactions that appeared on the blockchain within less than three hours between each other; there are no edges connecting the different time steps.

The first 94 features represent local information about the transaction – including the time step described above, number of inputs/outputs, transaction fee, output volume and aggregated figures such as average BTC received (spent) by the inputs/outputs and average number of incoming (outgoing) transactions associated with the inputs/outputs. The remaining 72 features are aggregated features, obtained using transaction information one-hop backward/forward from the center node - giving the maximum, minimum, standard deviation and correlation coefficients of the neighbour transactions for the same information data (number of inputs/outputs, transaction fee, etc.).

In [ ]:
# Mount the libraries:
# igraph
!pip install igraph

# Shortest path
!pip install cairocffi

# Role Embeddings
!pip install node2vec

# topological features
!pip install gudhi

# Ripser
!pip -q install ripser persim

# Standard
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# more matplotlib
import matplotlib.cm as cm
from matplotlib.lines import Line2D
import matplotlib.lines as mlines
from matplotlib.patches import Patch

# 3D
from mpl_toolkits.mplot3d import Axes3D

# Google Colab and files
from google.colab import files

# Runtime
import psutil, time

# Animate, batches, HTML and base
from IPython.display import HTML
from base64 import b64encode
import matplotlib.animation as animation
import matplotlib.patches as mpatches

# Graph
import networkx as nx

# Ticker
from matplotlib.ticker import FuncFormatter

# Eigenvalue and eigenvectors4
# Uses ARPACK under the hood in non-symmetric mode
from scipy.sparse.linalg import eigs
# Uses ARPACK in symmetric mode or Hermitian.
from scipy.sparse.linalg import eigsh

# K Means and Counter
from sklearn.cluster import KMeans
from collections import Counter, deque

# Betweenness Centrality
from igraph import Graph, plot
from networkx.algorithms.centrality import betweenness_centrality_subset

# hubs
from networkx.algorithms import cycle_basis

# Louvain
import community.community_louvain as community_louvain
import matplotlib.cm as cm

# Shortest path
import igraph as ig
from tqdm import tqdm
import glob
from math import log

# Role Embeddings
from node2vec import Node2Vec

# Simplicial Complex, Clique Complex, Polygon, Patch
import gudhi
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
import math
from typing import Dict, List, Tuple


# Ripser
from ripser import ripser

# operating system
import os, json

# garbage collector
import gc

# combinations of elements
from itertools import combinations, count

# Saving
import pickle

# Requires pairwise distances (e.g. shortest path)
from scipy.spatial.distance import squareform
from gudhi import RipsComplex

# Principal Component analysis (PCA)
from sklearn.decomposition import PCA

# Nearest Neighbors
from sklearn.neighbors import NearestNeighbors

# Standard Scaler or Robust
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

# Breadth First Search
from networkx.algorithms.traversal.breadth_first_search import bfs_edges

# Scaling
from sklearn.impute import SimpleImputer

# Log Regression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Metrics, Confustion Matrix
from sklearn.metrics import (classification_report,
confusion_matrix,
ConfusionMatrixDisplay,
f1_score, precision_score, recall_score, log_loss, precision_recall_curve,
roc_auc_score, roc_curve, brier_score_loss)

# Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Neural Network
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.neural_network import MLPClassifier

# Tools
from itertools import product

# Model and Pipeline
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline

# Function Tools reduce
from functools import reduce

# Path
from pathlib import Path

# Calibrated Classifier of OOF
from sklearn.calibration import CalibratedClassifierCV

In [ ]:
# Mount the data sets
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 3 data sets: Content, nodes and edges, and features.
df_classes = pd.read_csv('/content/drive/MyDrive/elliptic_txs_classes.csv')
df_edgelist = pd.read_csv('/content/drive/MyDrive/elliptic_txs_edgelist.csv')
# need a header
df_features = pd.read_csv('/content/drive/MyDrive/elliptic_txs_features.csv', header=None)


# Act 1 - Graph Theory

## Data for Classes

This is nodes and then either unknown illicit or licit.

In [ ]:
df_classes

In [ ]:
# How many types?
df_classes['class'].unique()

In [ ]:
# How many types and count?
df_classes['class'].value_counts()

In [ ]:
# How many classes and want percent?
df_classes['class'].value_counts(1)

The ID is all unique and there is a class is 'unknown', '2' and '1' and (4,545) of the nodes are labelled class1 (illicit) and (42,019) are labelled class2 (licit).

In [ ]:
df_classes.dtypes

In [ ]:
# '1' and '2' are objects so let's convert it with numerical pd.to_numeric
df_classes['class'] = pd.to_numeric(df_classes['class'], errors='coerce')

# using .map for 1-illict and 2-licit
# Also just leave the 'unknown' using .fillna
df_classes['class'] = df_classes['class'].map({1: "illicit", 2: "licit"}).fillna("unknown")

In [ ]:
df_classes

We filter and then print only illicit.

In [ ]:
# Make sure 'class' column is a string
df_classes['class'] = df_classes['class'].astype(str)

# Filter to get only the illicit nodes (class = 'illicit')
illicit_nodes = df_classes[df_classes['class'] == 'illicit']

# Optionally view the top 5
print(illicit_nodes.head())

Plot a visual using unknown, illict, lict.

In [ ]:
# visual for df_classes using values counts().plot, and kind='bar'
# illicit = red, and licit = green, unknow = gray

df_classes['class'].value_counts().plot(kind='bar',
                                        title='Class Distribution',
                                        color = ['gray', 'green', 'red'])
plt.xlabel('Class')
plt.ylabel('Count')
plt.show()

Again very small with `illicit 2.2%`, `licit 20.6%` and `unknown 77.1%`. `Unknown` Is shy of 160,000 or (157,205 exactly) `licit` Is a little bit more 40,000 or (42,019 exactly), and `illicit` is mabye 5,000 or (4545 exactly).

## Data for Edge List

Node directed edge to another node

In [ ]:
df_edgelist

In [ ]:
# How many columns?
df_edgelist.columns

In [ ]:
# Let's change the features from txId1 is the source and txId2 is the target
df_edgelist = df_edgelist.rename(columns = {'txId1':'source', 'txId2':'target'})

In [ ]:
# How many columns and new type
df_edgelist.columns

So just want it a little bit cleaner source to `source` versus `txId1` and `target` versus `txId2`. So that's the node is a source and the pointing to a target which is a direct graph of an edge.

## Data for Features

Here is data `df_features`.

In [ ]:
df_features

Let to a `txId` and a `time_step` is the data set, So that we can `df_classes` and `df_edgelist` Have the same nodes.

In [ ]:
# add header re-names columns
new_column_names = {
    0: 'txId',
    1: 'time_step',
    **{i: f'local_info_{i-1}' for i in range(2, 95)},
    **{i: f'agg_info_{i-94}' for i in range(95, 167)}
}

df_features = df_features.rename(columns=new_column_names)

In [ ]:
df_features

OK let's do the time series from 1 to 49 and Put a picture so we can see every step.

See [9, p 3]

In [ ]:
# Make sure txId is aligned (same dtype)
df_features['txId'] = df_features['txId'].astype(str)
df_classes['txId'] = df_classes['txId'].astype(str)

# Merge and keep labels
df_merged = df_features.merge(df_classes[['txId', 'class']], on='txId', how='left')

# Define the cleaning function
def clean_scores(df_features, df_classes):
    # Ensure consistent txId types
    df_features['txId'] = df_features['txId'].astype(str)
    df_classes['txId'] = df_classes['txId'].astype(str)

    # Merge labels into features
    df_scores = df_features.merge(df_classes[['txId', 'class']], on='txId', how='left')

    # Rename 'class' to 'label'
    df_scores.rename(columns={'class': 'label'}, inplace=True)

    # Drop rows with any missing features (not in df_features)
    feature_cols = df_features.columns.drop('txId')
    df_scores = df_scores.dropna(subset=feature_cols)

    return df_scores.reset_index(drop=True)

# Plot counts
counts = df_merged.groupby(['time_step', 'class']).size().unstack(fill_value=0)
counts = counts.reindex(columns=['unknown', 'licit', 'illicit'], fill_value=0)

# Stacked bar chart and color
custom_colors = ['#999999', '#2ca02c', '#d62728']
ax = counts.plot(kind='bar', stacked=True, figsize=(14, 6), color=custom_colors, width=0.9)

# plot title, xlable ect
plt.title('Transaction Count per Time Step by Class')
plt.xlabel('Time Step')
plt.ylabel('Number of Transactions')
plt.xticks(ticks=range(len(counts.index)), labels=counts.index, rotation=45)
plt.legend(title='Class')
plt.tight_layout()
plt.grid(True, axis='y', linestyle='--', alpha=0.5)
plt.show()


In [ ]:
# Visualize correlation within local features
# Select local info columns (features 2-94 based on description, index 1-93 in 0-indexed DataFrame)
local_info_features = df_features.iloc[:, 2:95]
plt.figure(figsize=(10, 8))
sns.heatmap(local_info_features.corr(), cmap='coolwarm', center=0)
plt.title("Correlation Heatmap — Local Transaction Features")
plt.show()

# Visualize correlation within aggregated features
# Select aggregated features (features 95-166 based on description, index 94-165 in 0-indexed DataFrame)
agg_features = df_features.iloc[:, 95:167]
plt.figure(figsize=(10, 8))
sns.heatmap(agg_features.corr(), cmap='coolwarm', center=0)
plt.title("Correlation Heatmap — 1-Hop Aggregated Features")
plt.show()

Not much correlation, especially when the `local_info` may have the same information, just a different way.

## Let's do it Animated time series.

So have to do Make sure that the string is consistency for all `df_features`, `df_edgelist`, and `df_classes`. This is a directed graph and using the `licit`, `illicit` and `unknown`.

In [ ]:
# Convert txId to string for consistency
df_features['txId'] = df_features['txId'].astype(str)
df_edgelist = df_edgelist.astype(str)
df_classes['txId'] = df_classes['txId'].astype(str)

# Build the full directed graph
G_directed = nx.DiGraph()
G_directed.add_edges_from(df_edgelist.values)

# Create a mapping of txId → class
class_map = dict(zip(df_classes['txId'], df_classes['class']))

Using subgraph for all from 1 to 49 timesteps.

In [ ]:
# df_features groupby time_step
dfs_by_time = {t: df for t, df in df_features.groupby("time_step")}

# tolist and subgraph
graphs_by_time = {}
for t, df_t in dfs_by_time.items():
    tx_ids = df_t['txId'].tolist()
    subG = G_directed.subgraph(tx_ids)
    graphs_by_time[t] = subG

Layout and color:

In [ ]:
# sprint layout
pos = nx.spring_layout(graphs_by_time[49], seed=42)

# color
def get_color(node):
    label = class_map.get(node, 'unknown')
    if label == 'illicit':
        return '#d62728'  # red
    elif label == 'licit':
        return '#2ca02c'  # green
    else:              # unknown
        return '#999999'  # gray

In [ ]:
# Update 1 to 49
def update(t):
    ax.clear()
    G_t = graphs_by_time[t]

    pos = nx.spring_layout(G_t, seed=42)

    colors = [get_color(n) for n in G_t.nodes()]

    # Add this to the bottom of your `update(t)` function:
    red_patch = mpatches.Patch(color='#d62728', label='Illicit')
    green_patch = mpatches.Patch(color='#2ca02c', label='Licit')
    gray_patch = mpatches.Patch(color='#999999', label='Unknown')
    ax.legend(handles=[red_patch, green_patch, gray_patch], loc='upper right', fontsize=10)

    nx.draw(
        G_t,
        pos=pos,
        ax=ax,
        node_size=15,
        node_color=colors,
        with_labels=False,
        edge_color="#cccccc",
        alpha=0.9
    )
    ax.set_title(f"Elliptic Transaction Graph — Time Step {t}", fontsize=16)

In [ ]:

fig, ax = plt.subplots(figsize=(10, 7))
ani = animation.FuncAnimation(fig, update, frames=range(1, 50), interval=600)
ani.save("elliptic_graph_colored.mp4", writer="ffmpeg", dpi=150)


# 1 hr

In [ ]:
# video
mp4 = open("elliptic_graph_colored.mp4", "rb").read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

HTML(f"""
<video width=600 controls>
    <source src="{data_url}" type="video/mp4">
</video>
""")

# 2 hr

### $k$-Hop Neighborhoods and Illicit Subgraphs

What is a $k$-hop neighborhood?

Let $n_0$ be a node of interest — for example, a node suspected of illicit activity. The $k$-hop neighborhood of $n_0$, denoted $N_k(n_0)$, is the set of all nodes that can be reached from $n_0$ by traversing at most $k$ edges.

When $k = 0$, the neighborhood includes only $n_0$ itself.

When $k=1$, we include all immediate neighbors connected by one edge.

When $k=2$, we include neighbors of neighbors — and so on.

You can think of this visually as concentric rings expanding out from the center node $n_0$. Each ring represents a further layer in the network, moving outward edge by edge.

So let’s focus only on illicit nodes

In [ ]:
# Make sure 'class' column is a string
df_classes['class'] = df_classes['class'].astype(str)

# Filter to get only the illicit nodes (class = 'illicit')
illicit_nodes = df_classes[df_classes['class'] == 'illicit']

# view the .head()
print(illicit_nodes.head())

We are using unweighted shortest path algorithms, which rely on **Breadth-First Search** or **(BFS)**. If edge weights were available or relevant, we would instead use **Dijkstra’s** algorithm to account for edge costs.

We focus on 5-hop neighborhoods meaning that $N_5(n_0)$ reachable within five steps or fewer. This captures the local structure and potential influence zone of a node without needing to traverse the entire network.

The algorithm effectively builds a **shortest-path spanning tree** rooted at the **center**, capturing the **minimal paths** to all nodes within the hop limit.

See [10, p. 99], [5, p. 315], [11, p. 3] and [13, p. 35].

In [ ]:
# Make sure that Consistently across both df_edgelist and df_classes:
df_edgelist['source'] = df_edgelist['source'].astype(str)
df_edgelist['target'] = df_edgelist['target'].astype(str)
df_classes['txId'] = df_classes['txId'].astype(str)

# Build the directed graph from edgelist
G_directed = nx.from_pandas_edgelist(df_edgelist, source='source', target='target', create_using=nx.DiGraph())

# Get a list of illicit nodes
illicit_nodes = df_classes[df_classes['class'] == 'illicit']['txId'].tolist()

# This loop ensures the selected start_node is: Illicit Actually present in the graph
# Pick an Illicit Node That Exists in the Graph
start_node = None
for node in illicit_nodes:
    if node in G_directed:
        start_node = node
        break

if start_node is None:
    print("No illicit nodes found in the graph.")
else:
    # Perform 5-hop neighborhood expansion
    # This function finds all nodes that are reachable from the start node within depth hops (in both directions).
    # `frontier` is the current boundary of the search.
    # `successors` = nodes that receive an edge from the current node (->)
    # `predecessors` = nodes that send an edge to the current node (<-)
    def get_n_hop_neighbors(graph, start, depth=5):
        visited = set([start])
        frontier = set([start])
        for _ in range(depth):
            next_frontier = set()
            for node in frontier:
                next_frontier.update(set(graph.successors(node)))
                next_frontier.update(set(graph.predecessors(node)))
            frontier = next_frontier - visited
            visited.update(frontier)
        return visited

    # Get subgraph
    # five_hop_nodes is a set of nodes found within 5 hops.
    # H is a new subgraph built from those nodes plus the original start_node.
    five_hop_nodes = get_n_hop_neighbors(G_directed, start_node, depth=5)
    H = G_directed.subgraph(five_hop_nodes | {start_node})  # include start node explicitly

    # Build color mapping
    node_colors = []
    class_map = df_classes.set_index('txId')['class'].to_dict()

    # This loop assigns:
    # Red for 'illicit'
    # Green for 'licit'
    # Gray if the class is unknown
    for node in H.nodes():
        cls = class_map.get(node, 'unknown')
        if cls == 'illicit':
            node_colors.append('red')
        elif cls == 'licit':
            node_colors.append('green')
        else:
            node_colors.append('gray')

    # Plot the colored subgraph
    # nx.kamada_kawai_layout() — like spring layout but more geometric
    # nx.circular_layout() — nodes in a circle
    # nx.shell_layout()

    plt.figure(figsize=(6, 3))
    pos = nx.shell_layout(H)  # layout for consistent spacing
    nx.draw(H, pos, node_color=node_colors, with_labels=False, node_size=30, alpha=0.8, arrows=True)
    plt.title(f"5-Hop Neighborhood from Illicit Node {start_node}")
    plt.show()

So you can see two illicit nodes are connected to the same licit node — almost like they're targeting it. But is that really two separate events, or could it still be coming from one main actor?

Let’s run a 3-hop expansion on a few more illicit nodes — maybe 6 in total. We’ll check whether each one connects to just a single target or multiple targets, and whether the patterns repeat or vary. This might help us understand if these illicit nodes are working together or acting independently.

In [ ]:
# Function: 3-hop neighborhood expansion
def get_n_hop_neighbors(graph, start, depth=3):
    visited = set([start])
    frontier = set([start])
    for _ in range(depth):
        next_frontier = set()
        for node in frontier:
            next_frontier.update(set(graph.successors(node)))
            next_frontier.update(set(graph.predecessors(node)))
        frontier = next_frontier - visited
        visited.update(frontier)
    return visited

# Color mapping helper
class_map = df_classes.set_index('txId')['class'].to_dict()

# Loop through first 6 illicit nodes that exist in G
count = 0
# Get the list of illicit transaction IDs
illicit_tx_ids = df_classes[df_classes['class'] == 'illicit']['txId'].tolist()

# Iterate through the illicit transaction IDs
for node in illicit_tx_ids:
    if node not in G_directed:
        continue

    start_node = node
    count += 1
    print(f"\n Visualizing 3-hop neighborhood for illicit node {start_node}")

    # Subgraph creation
    three_hop_nodes = get_n_hop_neighbors(G_directed, start_node, depth=3)
    H = G_directed.subgraph(three_hop_nodes | {start_node})

    # Node color mapping
    node_colors = []
    for n in H.nodes():
        cls = class_map.get(n, 'unknown')
        if cls == 'illicit':
            node_colors.append('red')
        elif cls == 'licit':
            node_colors.append('green')
        else:
            node_colors.append('gray')

    # Plot
    plt.figure(figsize=(6, 3))
    pos = nx.shell_layout(H)
    nx.draw(H, pos, node_color=node_colors, with_labels=False, node_size=30, alpha=0.8, arrows=True)
    plt.title(f"3-Hop Neighborhood from Illicit Node {start_node}")
    plt.tight_layout()
    plt.show()

    # Stop after 6
    if count >= 6:
        break

`232629023`: Same structure at 3 hops and 5 hops — two illicit nodes converging on one licit. Could suggest repeated coordination or one main actor.

`230389796`: Three illicit nodes connect to distinct licit nodes nearby, forming a cluster — this resembles a potential fraud ring.

`17387772`: Almost identical to `230389796` — the structural overlap suggests it may be part of the same coordinated group.

`232947878`: More chaotic, scattered connections — less like targeted fraud, more like background laundering.

`16754007`: One illicit node in a dense cluster of licit/unknown — could be a “hiding” strategy.

`231990430`: Same repeat pattern as `232629023` — two illicit nodes targeting one licit. Likely not coincidence.

## Degree Distribution in an Undirected Graph

A degree distribution represents how the degrees (number of edges) are **spread across all nodes** in a graph.

For an undirected graph with $N$ total nodes, we count how many nodes have degree $(0, 1, \dots, N-1)$ and gives us a **frequency distribution** of degrees.

To convert this into a **probability distribution**, we divide each frequency by $N$, so the total area under the distribution **sums to 1**. This gives us insight into the overall structure of the network

See [10, p. 243] and [4 , p. 6].

Also, for the code, I will using a `dataframe` called `scores`. That is useful for engineer features, And that will be throughout this Jupiter notebook especially when we start modeling.

In [ ]:
# create and undirected
G_undirected = nx.from_pandas_edgelist(df_edgelist, source='source', target='target', create_using=nx.Graph())

# Add degrees to your dataframe
# Add a scores DataFrame, Which we are going to use that eventually for modeling and predicting

df_scores = pd.DataFrame({
    'txId': list(G_undirected.nodes),
    'label': [class_map.get(n, 'unknown') for n in G_undirected.nodes]
})

# Add a scores to:
df_scores['degree'] = df_scores['txId'].map(lambda n: G_undirected.degree(n))

# split know classes
licit_deg = df_scores[df_scores['label'] == 'licit']['degree']
illicit_deg = df_scores[df_scores['label'] == 'illicit']['degree']
unknown_deg = df_scores[df_scores['label'] == 'unknown']['degree']

# Plot KDE (Kernel Density Estimation) for smooth curves
# using log1p Because it's very steep and then almost zero when degree gets larger
plt.figure(figsize=(8, 4))
sns.kdeplot(np.log1p(licit_deg), label='Licit', color='green', fill=True)
sns.kdeplot(np.log1p(illicit_deg), label='Illicit', color='red', fill=True)
sns.kdeplot(np.log1p(unknown_deg), label='Unknown', color='gray', fill=True)

# Set log-transformed x-axis ticks back to real degree values
plt.gca().xaxis.set_major_formatter(FuncFormatter(lambda x, _: f"{np.expm1(x):.0f}"))

# Set labels and
plt.xlabel("Degree")
plt.ylabel("Density")
plt.title("Distribution of Node Degree: Licit vs Illicit")
plt.legend()
plt.show()

To better understand the structure of the graph, I applied the `log1p` transformation to the node degrees. This helps visualize the degree density, especially since the distribution is highly skewed.

Most nodes have a degree of 1, 2, or 3, and then the frequency drops off sharply. However, some nodes exhibit degrees as high as 400 or more.

When comparing node classes:

`Illicit` and `licit` nodes both tend to have a peak at degree 1, followed by a **rapid decline**.

Interestingly, unknown nodes have a higher concentration at degree 2, which stands out and may suggest a different interaction pattern.

Note that this analysis is based on the total degree $(\text{in} + \text{out})$ in the undirected version of the graph — not separating in-degree from out-degree.

For further analysis, I stored the degree values as a new feature column in the `df_scores`.

In [ ]:
df_scores.info()

## Cycles

**Definition:**

Let a $G = (V,E)$ where $V$ is a **vertices** or a **node**, and $E$ are **edges**. A **simple graph** is a pair $G = (V,E)$ with $V \neq \emptyset$ and
$$ E \subseteq \{ \{u,v\} : u, v \in V, u \neq v\} $$

so edges are unordered pairs of distinct vertices (no loops, no parallel edges). $G$ is connected iff $\forall v_i \neq v_j \in V$ there exists a path $ v_i \rightsquigarrow v_j$; equivalently, $G$ has exactly one connected component.

A **path** is a listing of vertices what the starting point $v_1$ is set $ v_1, v_2, \dots , v_k $

A **cycle** in a graph is a path that starts and ends at the same node, with no repeated nodes $ v_1 \rightarrow v_2 \rightarrow \dots \rightarrow v_k \rightarrow v_1$ That each the nodes are placed around the circle, and the **order** does matter.

In an **undirected** graph, it's the same idea but edge direction doesn’t matter.

Many money laundering schemes rely on cycles to disguise the origin of funds. These circular transaction patterns can help money flow through multiple nodes and eventually re-enter the system appearing legitimate — making them harder to detect.

Let’s explore whether these cycles might reveal potential entry points for illicit activity.

Code:

We examine cycles in a directed graph and keep only those that contain enough `illicit` nodes. We’ll inspect up to six candidate start nodes. Let `min_illicit` be the minimum number of illicit nodes a cycle must contain to be considered interesting. `class_map` maps each node ID to its label ('illicit', 'licit', or 'unknown'). For any cycle, `red_count` is the number of nodes labeled 'illicit'. We filter cycles by `red_count >= min_illicit`. Then we ensure we’re evaluating cycles in the directed graph and log the results.

In [ ]:
# Runs a loops analysis on those same 6 nodes: `232629023` ect.

# Number of Illicit starting points to check and runtime control
max_nodes = 6

# list to store results
cycle_results = []

# Set threshold for what counts as "suspicious"
min_illicit = 1

# This function checks if a cycle contains enough illicit nodes (at least min_illicit).
# class_map.get(node) looks up the label ('illicit', 'licit', and unknowns).
# Cycles with enough red_count are considered interesting.
def is_interesting_cycle(cycle, class_map, min_illicit=1):
    red_count = sum(1 for node in cycle if class_map.get(node) == 'illicit')
    return red_count >= min_illicit

# Looping through list of known illicit node IDs (illicit_tx_ids)
# Skip nodes not in the graph (some might be missing from edge list)
count = 0
for node in illicit_tx_ids:
    if node not in G_directed:
        continue

    # Log the progress
    print(f"\n Analyzing 3-hop neighborhood for illicit node {node}")
    count += 1

    # 3-hop subgraph
    # get_n_hop_neighbors() returns all nodes reachable from node within 3 hops
    # H is a small subgraph centered on this node’s local neighborhood
    neighbors = get_n_hop_neighbors(G_directed, node, depth=3)
    H = G_directed.subgraph(neighbors | {node})

    # Finds simple directed cycles in the 3-hop neighborhood using NetworkX.
    # These are cycles with no repeated nodes except the start/end.
    cycles = list(nx.simple_cycles(H))
    interesting_cycles = [c for c in cycles if is_interesting_cycle(c, class_map, min_illicit=1)]

    # log the cycle_results
    if interesting_cycles:
        print(f"Found {len(interesting_cycles)} interesting cycle(s) in neighborhood of node {node}")
        for c in interesting_cycles:
            print(" -", c)
        cycle_results.append((node, interesting_cycles))
    else:
        print("No interesting cycles found.")
    # Stopping after 6 nodes
    if count >= max_nodes:
        break


So nothing yet And that's fine because some of the fraud are rare. The next is using Only illicit nodes using a directed subgraph.

In [ ]:
# Get all illicit node IDs using list and set()
illicit_nodes = df_classes[df_classes['class'] == 'illicit']['txId'].tolist()
illicit_nodes = set(illicit_nodes)

# Create subgraph of illicit-only nodes (including only edges between them)
G_illicit = G_directed.subgraph(illicit_nodes).copy()

#  Find all simple cycles in this subgraph
cycles = list(nx.simple_cycles(G_illicit))

# Print summary
print(f"Found {len(cycles)} cycles among illicit nodes only.")

# Arbitrate numbers for 4545
for c in cycles[:4546]:
    print("Cycle:", c)

Nothing there.

In [ ]:
# Start with illicit node IDs
illicit_nodes = set(df_classes[df_classes['class'] == 'illicit']['txId'].tolist())

# Find all neighbors (in or out) of illicit nodes
neighbors = set()
for node in illicit_nodes:
    if node in G_directed:
        neighbors.update(G_directed.successors(node))
        neighbors.update(G_directed.predecessors(node))

# Union to get an expanded subgraph
expanded_nodes = illicit_nodes.union(neighbors)
G_expanded = G_directed.subgraph(expanded_nodes).copy()

# Step 4: Find all simple cycles
cycles = list(nx.simple_cycles(G_expanded))

# Step 5: Filter cycles that contain 2+ illicit nodes
def is_interesting_cycle(cycle, class_map):
    return sum(1 for node in cycle if class_map.get(node) == 'illicit') >= 2 # illicit 1 ?

class_map = df_classes.set_index('txId')['class'].to_dict()
interesting_cycles = [c for c in cycles if is_interesting_cycle(c, class_map)]

# Step 6: Print results
print(f"Found {len(interesting_cycles)} interesting cycles with 2+ illicit nodes.")

# Arbitrate numbers for 4545
for c in interesting_cycles[:4545]:
    print("Cycle:", c)

Again nothing.

In [ ]:
print(f'Number of cycles: {len(cycles)}')

So, Even two elicited node of a corrected graph is still zero. So let's using undirected graph and see if that'll help.


### Change to directed to undirected.

We switch to an undirected graph $G = (V,E)$ and perform a **depth‑first search (DFS)** to build a search tree $T$.

Start from a root $u$ and grow $T$ by exploring unseen vertices.

When we encounter an edge $\{ {v,w} \}$:

If $w \notin V(T)$, add $\{ {v,w} \}$ to $T$ (a tree edge).

If $w \in V(T)$ and $w$ is an ancestor of $v$ in $T$, then $ \{ {v,w} \}$ is a back edge. The unique path in $T$ from $w$ to $v$, together with $\{ {v,w} \}$, forms a simple cycle.

After DFS (including **backtracking**), the set of cycles discovered from all back edges forms a cycle basis of $G$ (a minimal set of cycles from which every cycle in $G$ can be formed via symmetric difference of edges). See West [10, pg 156].

`networkx.cycle_basis(G)` implements this idea for undirected graphs, and the algirithm is Still today. See [17] K. Paton, *An algorithm for finding a fundamental set of cycles of a graph*.

In [ ]:
# undierected cycles
undirected_cycles = nx.cycle_basis(G_undirected)
print(f"Found {len(undirected_cycles)} undirected cycles in the full graph.")

OK now getting somewhere.

In [ ]:
suspicious_cycles = [c for c in undirected_cycles if any(class_map.get(n) == 'illicit' for n in c)]
print(f"Found {len(suspicious_cycles)} undirected cycles with at least one illicit node.")

So this is one node for `illicit` per a cycle, and the cycle with count and Produce the top score.

In [ ]:
illicit_counts_per_loop = [
    sum(1 for node in c if class_map.get(node) == 'illicit')
    for c in suspicious_cycles
]

illicit_cycle_count = Counter(illicit_counts_per_loop)
print("Illicit node count per cycle:")
for num_illicit, count in sorted(illicit_cycle_count.items()):
    print(f"{num_illicit} illicit nodes: {count} loops")

### Most suspicious unkowns nodes

Let a undirected cycle in the graph, and let $V = \text{nodes}$ Have at least 5 illicit nodes. We define (tau) **threshold** is bigger than 5. In other words: $\tau \geq 5$. We define all the cycles bigger than or equal to are $$ \mathcal C_{\tau}(G) = \{ C \in C(G) \mid \mathcal I(C) \geq 5 \} $$

where $C(G)$ meaning cycles are in $G$ and $\mathcal I (C)$ meaning that only looks through the threshold from cycle.


In [ ]:
suspicious_unknowns = Counter()

for cycle in suspicious_cycles:
    illicit_count = sum(1 for node in cycle if class_map.get(node) == 'illicit')
    if illicit_count >= 5:  # threshold
        for node in cycle:
            if class_map.get(node) == 'unknown':
                suspicious_unknowns[node] += 1

print("Most suspicious unknown nodes:")
print(suspicious_unknowns.most_common())

This ones will do those five illicit and then loop through those cycles and then put a plot so you can see what's going on.

In [ ]:
# Top 5 suspicious unknowns
top_unknowns = [node for node, _ in suspicious_unknowns.most_common(5)]

for i, unk_node in enumerate(top_unknowns):
    # Find cycles that include this unknown node
    relevant_cycles = [c for c in suspicious_cycles
                       if unk_node in c and sum(1 for n in c if class_map.get(n) == 'illicit') >= 5]

    # Flatten nodes and edges
    cycle_nodes = set(n for c in relevant_cycles for n in c)
    cycle_edges = [(c[j], c[(j+1)%len(c)]) for c in relevant_cycles for j in range(len(c))]

    # Build subgraph
    H = nx.Graph()
    H.add_edges_from(cycle_edges)

    # Node coloring
    node_colors = []
    for node in H.nodes():
        if node == unk_node:
            node_colors.append('blue')  # highlight target
        elif class_map.get(node) == 'illicit':
            node_colors.append('red')
        elif class_map.get(node) == 'licit':
            node_colors.append('green')
        else:
            node_colors.append('gray')

    # Plot
    plt.figure(figsize=(5, 4))
    pos = nx.spring_layout(H, seed=42)
    nx.draw(H, pos, node_color=node_colors, with_labels=False, node_size=120, edge_color='black')
    plt.title(f"Suspicious Unknown Node {unk_node} in {len(relevant_cycles)} cycles")
    plt.axis('off')
    plt.tight_layout()
    plt.show()

So this is confirmed and you can see on the visual the cycles are the same cycles just different nodes with suspicious illicit So that is a good insight for these cycles.

### Sub Graph Top 5 suspicious unknowns using eigenvalues, eigenvector and Spectral Clustering.

So for this we are going to use `nx.ego_graph` And what that means is  returns induced subgraph of neighbors centered at node $n$ within a given $\text{radius} = 3$ in the `def suspicious_none_report`.

This line creates a dictionary called class_map where:
Each key is a transaction ID (txId) and Each value is the corresponding class label ('illicit', 'licit', or 'unknown').

The code of `cycles = list(nx.cycle_basis(H))` is using **Johnson algorithm** using `networkx` in the hood.

See D. B. Johnson, *Finding all the elementary circuits of a directed graph* [12].

Next **Spectral Clustering** and **eigenvectors**

Under the hood we are using **ARPACK** or **ARnoldi PACKage**

The Arnoldi iteration for general (possibly non-symmetric) matrices

The Lanczos algorithm for symmetric (or Hermitian) matrices

These are both Krylov subspace methods that avoid computing the full matrix spectrum.

It’s all about finding a low-dimensional approximation of a giant matrix.

See Trefethen and Bau [7, Lecture 34]

Essentially what they're doing is how Arnoldi builds an orthonormal basis of the Krylov subspace, constructs a smaller matrix $H_k$ (Hessenberg form), and uses its eigenvalues as approximations to the original matrix’s eigenvalues.

That smaller matrix $H_k$ plays the same role as $T_k$ in the Lanczos method for symmetric matrices.

We use `from scipy.sparse.linalg import eigs`
and calling `ARPACK` (under the hood), which:

- Uses Arnoldi if the matrix is non-symmetric

- Uses Lanczos if it detects symmetry

ARPACK Is great because efficient, sparse-friendly, and backed by decades of numerical research.

Tiny little history:

- ISPACK – First eigenvalue solver library (1970s), written in FORTRAN.

- LINPACK – More general linear systems.

- LAPACK – Successor to both, with better performance and modern routines.

- ARPACK – Specializes in sparse eigenproblems using Arnoldi and Lanczos.

- SciPy / NumPy – Wraps all of these (through compiled C/Fortran bindings).

Python did not inveted the wheel, using `from scipy.sparse.linalg import eigs` calls ARPACK (written in FORTRAN) under the hood.

See Trefethen and Bau [7, Lecture 32]

In [ ]:
top_unknowns

In [ ]:
def suspicious_node_report(G_undirected, class_map, node_id, radius=3, show_plot=True, run_spectrum=True, n_clusters=2):
    node_id = str(node_id)

    # Build ego subgraph and H subgraph
    sub_nodes = nx.ego_graph(G_undirected, node_id, radius=radius)
    H = G_undirected.subgraph(sub_nodes)

    print(f"\n Suspicious Node: {node_id}")
    print(f"→ Subgraph size: {H.number_of_nodes()} nodes, {H.number_of_edges()} edges")

    # labels class and counts
    labels = [class_map.get(n, 'unknown') for n in H.nodes()]
    label_counts = Counter(labels)
    print("→ Label counts:", label_counts)

    # undirected or directed
    if not G_undirected.is_directed():
        cycles = list(nx.cycle_basis(H))
        print(f"→ Cycles found: {len(cycles)}")
    else:
        print("→ Skipping cycle detection (graph is directed)")

    # Visualize subgraph
    if show_plot:
        node_colors = []
        node_sizes = []
        for n in H.nodes():
            label = class_map.get(n, 'unknown')
            if n == node_id:
                node_colors.append('blue')
                node_sizes.append(150)
            elif label == 'illicit':
                node_colors.append('red')
                node_sizes.append(60)
            elif label == 'licit':
                node_colors.append('green')
                node_sizes.append(60)
            else:
                node_colors.append('gray')
                node_sizes.append(60)

        pos = nx.spring_layout(H, seed=42)
        nx.draw(H, pos, node_color=node_colors, node_size=node_sizes, with_labels=False, alpha=0.8)
        plt.title(f"{radius}-Hop Neighborhood of Suspicious Node {node_id}")
        plt.show()

    # Spectral Clustering and 50 eigen vaules and vectors
    if run_spectrum and H.number_of_nodes() > 5:
        try:
            A = nx.adjacency_matrix(H).astype(float)
            eigvals, eigvecs = eigs(A, k=min(50, H.number_of_nodes()-2))
            eigvals = np.real(eigvals)
            eigvecs = np.real(eigvecs)

            # Plot spectrum
            plt.figure(figsize=(6, 4))
            plt.plot(eigvals, 'bo-', label='Eigenvalues')
            plt.title(f"Eigenvalue Spectrum for Node {node_id}")
            plt.xlabel("Index")
            plt.ylabel("Eigenvalue")
            plt.grid(True)
            plt.legend()
            plt.show()

            # KMeans on top 50 eigenvectors
            coords = eigvecs[:, :2]
            kmeans = KMeans(n_clusters=n_clusters, n_init=10, random_state=42)
            cluster_labels = kmeans.fit_predict(coords)

            # Plot embedded 2D points
            plt.figure(figsize=(6, 5))
            plt.scatter(coords[:, 0], coords[:, 1], c=cluster_labels, cmap='viridis', s=60)
            plt.title(f"Spectral Clustering of Node {node_id}'s Neighborhood")
            plt.xlabel("1st Eigenvector")
            plt.ylabel("2nd Eigenvector")
            plt.grid(True)
            plt.show()

        except Exception as e:
            print(" Spectral analysis failed:", e)

In [ ]:
top_unknowns = ['71462499', '73224674', '72629376', '71461473', '29949646']

for uid in top_unknowns:
    suspicious_node_report(G_undirected, class_map, uid, radius=3, show_plot=True, run_spectrum=True)

So, What is going on? I'm going to pick node $1 \rightarrow$ `71462499` and node $4 \rightarrow$ `71461473`.

1. Graph Structure

Node $1$ have smaller ego network (fewer nodes and edges). Has a clear cycle with 5 illicit + 2 unknown nodes. Structure has loops + hubs, but the loops are relatively tight and localized. Many branches terminate quickly — fewer long-range interconnections. This compactness often means fewer large eigenvalues in the adjacency spectrum.

Node $4$

Much larger ego network (771 nodes, 970 edges). Multiple hubs and dense interconnections — network looks more meshed. The illicit nodes (red) form several large, separate clusters with strong connectivity to licit and unknown nodes. Far more edges per node on average, meaning higher degree variance and more diverse local structures.

2. Eigenvalue Spectrum

Node $1$

Eigenvalues between -4 and +4, tapering toward -1 to 0. This narrower range suggests fewer dominant structural patterns. The largest eigenvalues correspond to the cycle and its few main hubs.

Node $4$

Eigenvalues span ~ -10 to +10. Bigger magnitude eigenvalues indicate more strongly connected substructures and higher degree nodes. The large range + multiple spikes reflects several dense hubs and more global structure in the ego network.

3. Spectral Clustering Shape

Node $1$

First two eigenvectors produce a clear V-shape, indicating 2–3 well-separated communities in embedding space. Suggests a simpler community split — likely one illicit-heavy cycle and one licit-heavy hub.

Node $4$

Clustering plot shows tighter, overlapping points but with a few outliers far from the core. Means most of the network is strongly interconnected, but some smaller communities (outliers) exist. This is consistent with the “big hubs + small satellites” structure you see in the ego-graph.

4. Fraud-Detection Implications

Node $1$: Smaller, more defined illicit cycle — potentially a local fraud ring.

Node $4$: Large, hub-heavy network with mixed activity — potentially a central broker node connecting multiple illicit and licit subgroups, higher complexity, and potentially higher reach.



## Degree Centrality

According to Newman **degree centrality** measures how many direct connections a node has compared to the maximum possible.

For an undirected graph with $n$ nodes and $m$ edges the mean degree is

$$ c = \frac{1}{n} \sum_{i=1}^{n} k_i = \frac{2m}{n} $$
	​


where $k_i$ is the degree of node $i$. Normalizing by the maximum possible degree $(n−1)$ gives

$$ \rho = \frac{c}{n-1} , \rho \in [0,1] $$

This $\rho$ is also known as **graph density**—the fraction of realized edges out of all possible edges.

See Newman, *Networks: An Introduction* [5, p. 133].

Degree centrality is a simple but powerful indicator of how **connected** a node is in the network. In fraud analysis, unusually high degree nodes may act as hubs in illicit flows.



In [ ]:
# degree centrality
deg_centrality = nx.degree_centrality(G_undirected)

In [ ]:
df_scores = df_scores.merge(pd.DataFrame(deg_centrality.items(), columns=['txId', 'deg_centrality']), on='txId', how='left')

In [ ]:
# mean of degree centrality
mean_deg_centrality = sum(deg_centrality.values()) / len(deg_centrality)
print(f"Mean Degree Centrality: {mean_deg_centrality:.4f}")

# density \rho
density = nx.density(G_undirected)
print(f"Network Density: {density:.4f}")

In [ ]:
print("Nodes:", G_undirected.number_of_nodes())
print("Edges:", G_undirected.number_of_edges())

The last two code these are very **sparse**.


## Degree Distribution and Power Laws in Directed Graphs

### Background

We follow Newman's treatment (Section 8.4) of power-law degree distributions in real-world networks. Many systems, such as the World Wide Web, exhibit degree distributions of the form:


$$p_k = C k^{-\alpha} $$

This implies that most nodes have low degree, but a few nodes (hubs) have very high degree. Such distributions are called **scale-free** and appear linear on a log-log plot.

### In-Degree and Out-Degree Distributions

Since our transaction graph is **directed**, each node has:

- In-degree: number of incoming edges
- Out-degree: number of outgoing edges

We compute both distributions using NetworkX:

`in_degrees = dict(G.in_degree())`
and
`out_degrees = dict(G.out_degree())`

In [ ]:
# Calculate degrees
degrees = [degree for node, degree in G_undirected.degree()]

degree_counts = np.bincount(degrees)
degree_values = np.arange(len(degree_counts))

plt.figure(figsize=(6,4))
plt.loglog(degree_values[degree_counts > 0], degree_counts[degree_counts > 0], marker='o', linestyle='None')
plt.title("Log-Log Degree Distribution")
plt.xlabel("Degree")
plt.ylabel("Frequency")
plt.grid(True)
plt.show()

**Overall Degree Distribution** (log–log):

The Elliptic graph exhibits a heavy-tailed distribution, consistent with scale-free behavior.

In [ ]:
# Calculate in-degree and out-degree for each node
in_degrees = dict(G_directed.in_degree())
out_degrees = dict(G_directed.out_degree())


plt.figure(figsize=(8, 4))
sns.kdeplot(np.log1p(list(in_degrees.values())), label='In-degree', color='blue', fill=True)
sns.kdeplot(np.log1p(list(out_degrees.values())), label='Out-degree', color='orange', fill=True)

# Set log transformed x-axis ticks back to real on and out values
plt.gca().xaxis.set_major_formatter(FuncFormatter(lambda x, _: f"{np.expm1(x):.0f}"))

plt.title("Distribution of In-Degree and Out-Degree")
plt.legend()
plt.xlabel("Degree")
plt.ylabel("Frequency")
plt.title("In/Out Degree Distributions")
plt.show()

Here are the log log which power laws using of a straight line in the visual plots.

In [ ]:
def plot_log_log(degree_dict, title):
    count = Counter(degree_dict.values())
    x, y = zip(*sorted(count.items()))
    x = np.array(x)
    y = np.array(y) / sum(y)  # normalize to get p_k
    plt.loglog(x, y, marker='o', linestyle='None')
    plt.xlabel("k")
    plt.ylabel("p(k)")
    plt.title(title)
    plt.show()

plot_log_log(in_degrees, "In-Degree Distribution (log-log)")
plot_log_log(out_degrees, "Out-Degree Distribution (log-log)")

**In-Degree Distribution** (log–log):

Most transactions receive funds from only a few sources (low in-degree), while a small number of nodes receive funds from hundreds of sources. This **heavy-tailed** pattern suggests the presence of **hub** accounts that aggregate flows—potential candidates for fraud rings or laundering activity.

**Out-Degree Distribution** (log–log):

Similarly, most transactions send funds to only one or two neighbors, but a few nodes disperse funds to many addresses. These high out-degree **fan-out** nodes may represent distribution points in illicit activity, pushing money across many paths at once.

In [ ]:
# Build degree distribution (using undirected for simplicity here)
degrees = [d for n, d in G_undirected.degree()]
count = Counter(degrees)

# Extract x = degree values, y = frequencies (normalized as probabilities)
x, y = zip(*sorted(count.items()))
x = np.array(x)
y = np.array(y) / sum(y)   # probability distribution p(k)

# Fit linear regression in log space for rough slope estimate
mask = (x > 0) & (y > 0)
coeffs = np.polyfit(np.log10(x[mask]), np.log10(y[mask]), 1)
slope = coeffs[0]
print(f"Estimated slope (gamma) ~ {-slope:.2f}")

Exponent estimate. Fitting a line to the log–log degree distribution tail gives $ \alpha \approx 2.14 $ This sits in the typical range $ 2 < \alpha < 3 $ reported by Newman for many real-world networks, indicating a heavy tail with hubs but not so extreme that a few nodes absorb almost all connectivity.

See [5, p. 248]

## Betweenness Centrality

We compute node betweenness centrality on an undirected, unweighted graph using `igraph`’s implementation of the **Brandes** algorithm. In the unweighted case, shortest paths are obtained via **breadth-first search** (BFS); if edge weights are supplied, `igraph` switches to **Dijkstra’s** algorithm. Brandes then performs a dependency **back-propagation** step to accumulate, for each node, how often it lies on all-pairs shortest paths. The runtime is $O(nm)$ for unweighted graphs.

Nodes with high betweenness often act as bridges between communities. In fraud analysis, such nodes can represent **choke-points** or brokers that connect otherwise separated clusters of transactions.

See Brandes, *A Faster Algorithm for Betweenness Centrality* [13].

In [ ]:
# Brandes algorithm

# Build igraph graph and no weights
edges = list(zip(df_edgelist['source'], df_edgelist['target']))
g = Graph.TupleList(edges, directed=False)

# Compute betweenness
btw_scores = g.betweenness()
vertex_names = g.vs['name']

# Save to DataFrame
btw_df = pd.DataFrame({'txId': vertex_names, 'betweenness': btw_scores})

# 2. Merge betweenness
btw_df['txId'] = btw_df['txId'].astype(str)
df_scores = df_scores.merge(btw_df, on='txId', how='left')

Histogram of betweenness centrality and shows the skew of most nodes have very low betweenness, a few have very high.

In [ ]:
plt.figure(figsize=(6,4))
plt.hist(btw_scores, bins=100, color="steelblue", alpha=0.8)
plt.xlabel("Betweenness Centrality")
plt.ylabel("Number of Nodes")
plt.title("Distribution of Betweenness Centrality")
plt.yscale("log")  # often highly skewed, log scale helps
plt.show()

Most nodes rarely appear on shortest paths, but a small fraction have very high betweenness. These high-betweenness **bridges** may represent brokers or choke points in laundering flows.

In [ ]:
# Pick a node of interest (e.g. highest betweenness in full graph)
center_node = vertex_names[np.argmax(btw_scores)]

# Ego network: all nodes within 2 steps
ego_nodes = g.neighborhood(center_node, order=2)
subG = g.subgraph(ego_nodes)

# --- Prep sizes (betweenness) ---
btw_sub = np.array(subG.betweenness())
# robust scaling to avoid "all purple" effect
p5, p95 = np.percentile(btw_sub, [5, 95])
btw_scaled = np.clip((btw_sub - p5) / (p95 - p5 + 1e-9), 0, 1)
sizes = 6 + 20 * btw_scaled  # small base + emphasis on higher btw

# --- Color by class (objective-aligned) ---
# Build txId -> class map (licit/illicit/unknown) once earlier in your notebook
# class_map = dict(zip(df_classes['txId'].astype(str), df_classes['class']))

# igraph vertex names are strings
labels = [class_map.get(v['name'], 'unknown') for v in subG.vs]
color_map = {'illicit': '#d62728', 'licit': '#2ca02c', 'unknown': '#999999'}
colors = [color_map.get(lbl, '#999999') for lbl in labels]

layout = subG.layout("fr")
plot(
    subG,
    layout=layout,
    vertex_size=sizes,        # size encodes betweenness
    vertex_color=colors,      # color encodes class
    edge_color="lightgray",
    bbox=(700,700)
)

Betweenness x Class (ego network):

Node size encodes betweenness centrality; color encodes class (green=licit, red=illicit, gray=unknown). Large nodes act as structural bridges. Large gray or red nodes are especially interesting: they may represent chokepoints in laundering flows.

In [ ]:
top_btw = btw_df.sort_values("betweenness", ascending=False).head(10)
display(top_btw.merge(df_classes, on="txId"))

Top betweenness nodes. Most of the **highest-betweenness** nodes are labeled licit (likely exchanges/aggregators), but a few unknown nodes rank near the top. Unknown chokepoints are high-priority review candidates because they sit on many transaction paths and can serve as covert **brokers** in **laundering flows**.

## Eigenvector Centrality

Scores each node in proportion to the sum of its neighbors’ scores. In matrix form it’s the dominant eigenvector $x$ of the adjacency $A$ solving $ A x = \lambda x$. `NetworkX` uses **power iteration**:

$$ \mathbf x_{k+1} = \frac{\mathbf A \mathbf x_k}{\left\| \mathbf A \mathbf x_k \right\|} $$

until convergence.


Flags nodes embedded in well-connected cores (connected to other **central nodes**). In fraud graphs, these can be **ring leaders** or exchange-like **hubs—complements** degree (local) and betweenness (bridges).

See [7, p. 205]

In [ ]:
# compute eigenvector centrality
eig_centrality = nx.eigenvector_centrality(
    G_undirected,
    max_iter=5000,
    tol=1e-4
)

# convert to DataFrame
eig_centrality_df = pd.DataFrame({
    'txId': list(eig_centrality.keys()),
    'eigenvector': list(eig_centrality.values())
})

# merge into scores
eig_centrality_df['txId'] = eig_centrality_df['txId'].astype(str)
df_scores = df_scores.merge(eig_centrality_df, on='txId', how='left')

In [ ]:
# Subgraph to visualize: 2-hop ego of a top eigenvector node
top_node = max(eig_centrality, key=eig_centrality.get)
G_giant = G_undirected
ego = nx.ego_graph(G_giant, top_node, radius=2)

# Robust size scaling
vals = np.array([eig_centrality.get(n, 0.0) for n in ego.nodes()])
p5, p95 = np.percentile(vals, [5, 95])
sz = 50 + 400 * np.clip((vals - p5) / (p95 - p5 + 1e-12), 0, 1)

# Color by class
color_map = {'illicit': '#d62728', 'licit': '#2ca02c', 'unknown': '#999999'}
colors = [color_map.get(class_map.get(str(n), 'unknown'), '#999999') for n in ego.nodes()]

pos = nx.spring_layout(ego, seed=42)
plt.figure(figsize=(8,8))
nx.draw_networkx_edges(ego, pos, edge_color='lightgray', alpha=0.5, width=0.5)
nx.draw_networkx_nodes(ego, pos, node_color=colors, node_size=sz)
plt.title("Eigenvector Centrality × Class (ego network)")
plt.axis('off'); plt.show()

Eigenvector × Class (ego network). Node size ∝ eigenvector centrality (importance in a well-connected core); color encodes class (green=licit, red=illicit, gray=unknown). Large gray/red nodes near other large nodes are high-value review targets.

In [ ]:
sns.histplot(df_scores['eigenvector'], bins=100, log_scale=True)
plt.title("Distribution of Eigenvector Centrality")
plt.xlabel("Eigenvector Score")
plt.ylabel("Frequency")
plt.show()

Distribution is **heavy-tailed**: most nodes have tiny scores; a minority sit in highly central cores.

## Homophily and Assortative Mixing

We will use a 1 hop and compute `illicit` nodes on the top divided by 3 classes of a 1 hop nodes:

$$ r_i = \frac{\text{illicit neighbors}(n_i)}{\text{all neighbors}(n_i)} $$

where:

- $n_i$ is ego node, and only have illicit neighbors
- $\text{all neighbors}(n_i)$ is $\text{deg}(n_i)$ for all `'classes'` in the neighbors.
- $r_i$ is a ratio from $\text{illicit neighbors} / \text{all neighbors}$

This gives a local homophily signal you can feed into models.

In [ ]:
# % of direct neighbors that are illicit
def neighbor_illicit_ratio(node):
    neighbors = list(G_undirected.neighbors(node))
    illicit = sum(1 for n in neighbors if class_map.get(n) == 'illicit')
    return illicit / len(neighbors) if neighbors else 0

df_scores['neighbors_illicit_ratio'] = df_scores['txId'].map(neighbor_illicit_ratio).fillna(0)

Here is the top to bottom for the rows:

In [ ]:
df_scores[['label', 'neighbors_illicit_ratio']]

Here is the sort values top to the bottom:

In [ ]:
df_scores[['label', 'neighbors_illicit_ratio']].sort_values(by='neighbors_illicit_ratio', ascending=False)

Here it is a plot using a licit which a high probability of illicit.

In [ ]:
def plot_one_hop(G, center, class_map):
    H = nx.ego_graph(G, center, radius=1, undirected=True)
    labels = {n: class_map.get(n, 'unknown') for n in H.nodes()}

    color_map = {'illicit': 'red', 'licit': 'green', 'unknown': 'gray'}
    node_colors = [color_map.get(labels[n], 'gray') for n in H.nodes()]
    node_sizes  = [300 if n == center else 80 for n in H.nodes()]

    pos = nx.spring_layout(H, seed=0)
    plt.figure(figsize=(7,7))
    nx.draw_networkx_edges(H, pos, alpha=0.3)
    nx.draw_networkx_nodes(H, pos, node_color=node_colors, node_size=node_sizes)
    plt.axis('off')

    # stats for the title
    nbrs = [n for n in H.nodes() if n != center]
    cnt = Counter(labels[n] for n in nbrs)
    denom = len(nbrs)
    ratio = 0.0 if denom == 0 else cnt['illicit'] / denom
    plt.title(f"1-hop ego graph for {center}: illicit {cnt['illicit']}/{denom} = {ratio:.2f}")

    # legend
    legend_elems = [
        Line2D([0],[0], marker='o', color='w', label='illicit', markerfacecolor='red', markersize=8),
        Line2D([0],[0], marker='o', color='w', label='licit', markerfacecolor='green', markersize=8),
        Line2D([0],[0], marker='o', color='w', label='unknown', markerfacecolor='gray', markersize=8),
    ]
    plt.legend(handles=legend_elems, loc='best')
    plt.show()

# Find licit nodes with high illicit neighbor ratio,
# Ratio =< 0.85, 5 =< Neighborhood
cand_high = df_scores[
    (df_scores['label'] == 'licit') &
    (df_scores['neighbors_illicit_ratio'] >= 0.85) &
    (df_scores['degree'] >= 5)
].copy()

# grab some nodes to inspect
sample_nodes = cand_high['txId'].head(6).tolist()
print("Sample licit nodes with high illicit ratio:", sample_nodes)


# plot
for node_id in sample_nodes:
    plot_one_hop(G_undirected, node_id, class_map)


You can see that the node is the big ego and you can see node `179112629` has only one of $2$ `unknowns` and $27$ `illicit`.

Node `179113037` has $2$ `licit`, $3$ `unknowns` and $32$ `illicit`.

Definitely suspicious, And a lot of the same actors or multiple hits. These local homophily ratios highlight structural red flags: licit-labeled nodes embedded in predominantly illicit neighborhoods. Such nodes may indicate “front” accounts or compromised intermediaries, making them valuable for fraud detection models.

## PageRank

We computed PageRank on the **directed** transaction graph $G$, using the standard **damping factor** $\alpha = 0.85$, which corresponds to the **random surfer model**. Convergence was determined by setting the tolerance proportional to the number of nodes `(tolerance = |G| x tol)`.

PageRank was originally designed to evaluate the importance of web pages through their hyperlink structure, forming the basis of early search engines. In this context, we adapt the method to transaction networks, where high PageRank values may highlight influential or suspicious transactions.

See Page, Brin, Motwani, and Winograd, *The PageRank Citation Ranking: Bringing Order to the Web* [15] and Hagberg, Schult, and Swart, *Exploring network structure, dynamics, and function using NetworkX* [16].

In [ ]:
# Check G directed is the big graph for pagerank
try:
    G_directed
except NameError:
    raise ValueError("Please load your real Elliptic graph object `G` before continuing.")

# Compute PageRank on full graph
pagerank_scores = nx.pagerank(G_directed, alpha=0.85)

# Convert to DataFrame
pagerank_df = pd.DataFrame({
    'txId': list(pagerank_scores.keys()),
    'pagerank': list(pagerank_scores.values())
})

# Display top 10 PageRank nodes
top_pagerank = pagerank_df.sort_values(by='pagerank', ascending=False).head(10)

# Plot top 10
plt.figure(figsize=(10, 5))
plt.bar(top_pagerank['txId'].astype(str), top_pagerank['pagerank'], color='darkorange')
plt.title("Top 10 Nodes by PageRank (Elliptic Graph)")
plt.xlabel("Transaction ID")
plt.ylabel("PageRank Score")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# add scores.pd.DataFrame([])
df_scores = df_scores.merge(pagerank_df, on='txId', how='left')

Here is the top node; `163832295` and extracted its 3-hop neighborhood (both in- and out-edges).

In [ ]:
# Ensure consistent types
df_classes['txId'] = df_classes['txId'].astype(str)
class_map = dict(zip(df_classes['txId'], df_classes['class']))
target_node = str(top_pagerank['txId'].iloc[0])

# in and out:
def k_hop_nodes_bidirectional(G, node, k=3):
    UG = G.to_undirected()
    return set(nx.single_source_shortest_path_length(UG, node, cutoff=k).keys())

# Collect nodes within k hops (both directions), then induce directed subgraph
k = 3
nodes_k = k_hop_nodes_bidirectional(G_directed, target_node, k=k)
subG = G_directed.subgraph(nodes_k).copy()

# Build colors and sizes
def node_color(n):
    lab = class_map.get(str(n), 'unknown')
    return 'red' if lab == 'illicit' else ('green' if lab == 'licit' else 'gray')

colors = [node_color(n) for n in subG.nodes()]
sizes  = [220 if str(n) == target_node else 40 for n in subG.nodes()]

# Layout + draw
plt.figure(figsize=(9,9))
pos = nx.spring_layout(subG, k=0.15, seed=42)

nx.draw_networkx_nodes(subG, pos, node_color=colors, node_size=sizes)
nx.draw_networkx_edges(subG, pos, arrows=True, edge_color='lightgray', width=0.6, alpha=0.7, arrowsize=8)
# Optionally label just the ego
nx.draw_networkx_labels(subG, pos, labels={target_node: target_node}, font_size=8)

plt.title(f"{k}-Hop Neighborhood of Top PageRank Node {target_node} (directed edges)")
plt.axis('off')

# Simple legend

leg_items = [
    mlines.Line2D([], [], color='red', marker='o', linestyle='None', label='illicit'),
    mlines.Line2D([], [], color='green', marker='o', linestyle='None', label='licit'),
    mlines.Line2D([], [], color='gray', marker='o', linestyle='None', label='unknown'),

]
plt.legend(handles=leg_items, frameon=False, loc='upper left')
plt.show()

We observe that the node acts as a hub, connecting to clusters of both licit and illicit transactions. This mixture suggests that while PageRank emphasizes global influence, k-hop neighborhood analysis provides local signals that can be used as features in downstream models (e.g., fraction illicit within 3 hops).

## Suspicious Cycle Score

We compute a Suspicious Cycle Score using the `nx.cycle_basis(G_undirected)` function from `NetworkX`, which implements a classic algorithm for extracting a fundamental set of cycles from a spanning tree. Again, this method was first described by K. Paton in 1969, originally implemented in Fortran IV on IBM systems, and remains widely used today for efficient cycle detection [17].

We compute suspicious cycle scores using `nx.cycle_basis` and a cycle basis is not the set of all cycles but a minimal generating set. Different DFS roots may produce different—but equally valid—bases. Since our focus is on cycles enriched in illicit activity, we adopt Paton’s method for efficiency, while noting that alternative bases (e.g., the minimum cycle basis) exist.

In the context of fraud detection, cycles in transaction networks can indicate money laundering or circular flows of illicit funds. To adapt Paton’s approach, we identify cycles containing a high concentration of illicit transactions. Unknown transactions that appear within these cycles are flagged with higher scores, since their proximity to repeated illicit activity makes them more suspicious.

In [ ]:
# Initialize counter
suspicious_unknowns = Counter()

# Get all simple cycles (undirected)
undirected_cycles = nx.cycle_basis(G_undirected)

# Scan each cycle
for cycle in undirected_cycles:
    illicit_count = sum(1 for node in cycle if class_map.get(node) == 'illicit')
    if illicit_count >= 5:
        for node in cycle:
            if class_map.get(node) == 'unknown':
                suspicious_unknowns[node] += 1

# Add to scores
df_scores['suspicious_cycle_score'] = df_scores['txId'].map(suspicious_unknowns).fillna(0)

In [ ]:
# Filter for suspicious unknown nodes with high degree
suspicious_high_deg_unknowns = df_scores[
    (df_scores['label'] == 'unknown') &
    (df_scores['degree'] >= 100) &
    (df_scores['suspicious_cycle_score'] > 0)
]

# Show top suspicious unknowns
suspicious_high_deg_unknowns.sort_values(by='suspicious_cycle_score', ascending=False).head()

In [ ]:
target = str(355174807)

# Get 3-hop neighbors (BFS depth=3)
neighbors_3hop = nx.single_source_shortest_path_length(G_undirected, target, cutoff=3)
sub_nodes = list(neighbors_3hop.keys())
subgraph = G_undirected.subgraph(sub_nodes)

# Color by label
color_map = []
for node in subgraph:
    label = class_map.get(node, 'unknown')
    if label == 'illicit':
        color_map.append('red')
    elif label == 'licit':
        color_map.append('green')
    else:
        color_map.append('gray')

# Draw
plt.figure(figsize=(10, 10))
nx.draw_spring(subgraph, node_color=color_map, with_labels=False, node_size=30)
plt.title("3-Hop Neighborhood of Node 355174807")
plt.show()

That is definitely a hub!

In [ ]:
# This will give you all simple cycles in the **undirected** graph
all_cycles = cycle_basis(G_undirected)

target = str(355174807)

cycles_with_target = [
    c for c in all_cycles
    if target in c and sum(1 for n in c if class_map.get(n) == 'illicit') >= 5
]

print(f"Found {len(cycles_with_target)} illicit-heavy cycles involving node {target}")
for i, c in enumerate(cycles_with_target[:3]):  # Show just a few
    print(f"Cycle {i+1}: {c}")

In [ ]:
cycle_subgraph = G_undirected.subgraph(cycles_with_target[0])

color_map = [
    'red' if class_map.get(n) == 'illicit'
    else 'green' if class_map.get(n) == 'licit'
    else 'gray'
    for n in cycle_subgraph
]

plt.figure(figsize=(8, 8))
nx.draw_circular(cycle_subgraph, node_color=color_map, with_labels=True, node_size=300)
plt.title("Example Illicit-Heavy Cycle Involving Target Node")
plt.show()

## Louvain Community Detection

We applied Louvain clustering to partition the transaction graph into communities. The Louvain method is a greedy optimization algorithm that maximizes modularity, a measure of the density of edges inside a community compared to edges between communities. Originally introduced by Blondel et al. in 2008, it remains one of the most widely used methods for large-scale community detection due to its efficiency and scalability [18].

In fraud detection, community detection is valuable because illicit transactions often cluster together in hidden groups. By identifying communities with a high concentration of illicit activity, we can flag unknown transactions in the same cluster as potentially suspicious.

In [ ]:
# Run Louvain clustering on undirected graph
partition = community_louvain.best_partition(G_undirected)

# Get cluster of the suspicious node
target_cluster_id = partition.get(target)
target_cluster_nodes = [n for n, cid in partition.items() if cid == target_cluster_id]

# add as a feature
df_scores['louvain_cluster'] = df_scores['txId'].map(partition).fillna(-1).astype(int)

# Visualize that community
subgraph_cluster = G_undirected.subgraph(target_cluster_nodes)

# color
color_map = [
    'red' if class_map.get(n) == 'illicit'
    else 'green' if class_map.get(n) == 'licit'
    else 'gray'
    for n in subgraph_cluster
]

# plot
plt.figure(figsize=(10, 10))
nx.draw_spring(subgraph_cluster, node_color=color_map, with_labels=False, node_size=30)
plt.title(f"Louvain Cluster of Node {target}")
plt.show()

In [ ]:
# 1) Louvain partition (works on undirected)
partition = community_louvain.best_partition(G_undirected)

# 2) Pick the cluster for your target node
#    (Ensure target exists and types match your graph's node types.)
assert target in G_undirected, f"{target} not in graph"
target_cluster_id = partition.get(target)
target_cluster_nodes = [n for n, cid in partition.items() if cid == target_cluster_id]

# 3) Add cluster id as a feature (optional)
df_scores['louvain_cluster'] = df_scores['txId'].map(partition).fillna(-1).astype(int)

# 4) Build the subgraph
subgraph_cluster = G_undirected.subgraph(target_cluster_nodes).copy()

# 5) Make colors IN THE SAME ORDER as the nodes you’ll draw
nodes_in_order = list(subgraph_cluster.nodes())
def node_color(n):
    lbl = class_map.get(n, 'unknown')
    if lbl == 'illicit': return 'red'
    if lbl == 'licit':   return 'green'
    return 'gray'

color_map = [node_color(n) for n in nodes_in_order]

# 6) Layout + draw
plt.figure(figsize=(10, 10))
pos = nx.spring_layout(subgraph_cluster, seed=42)  # deterministic layout
nx.draw_networkx_nodes(subgraph_cluster, pos, nodelist=nodes_in_order,
                       node_color=color_map, node_size=18, alpha=0.9)
nx.draw_networkx_edges(subgraph_cluster, pos, width=0.3, alpha=0.4)
plt.title(f"Louvain Cluster of Node {target} (size={len(nodes_in_order)})")
plt.axis('off')
plt.show()

Louvain Cluster of Node 355174807

This cluster represents a large community of over 4,000 nodes detected using the Louvain algorithm. The structure is densely interconnected, with noticeable sub-clusters forming around illicit nodes (red). The mix of licit (green), illicit (red), and unknown (gray) nodes illustrates heavy traffic between all three classes, but the concentration of illicit nodes in localized “hotspots” suggests areas of heightened fraud activity within the broader community.

## Clustering Coefficient

The clustering coefficient measures the extent to which a node’s neighbors are connected to each other, forming local triangles in the network. Originally popularized by Watts and Strogatz in their small-world network model [19], the formal definition of local clustering and redundancy is detailed by Newman [5].

In transaction graphs, a high clustering coefficient may suggest collusion or tightly organized fraud rings. Conversely, a low coefficient may indicate more random or opportunistic transactions. Incorporating this metric provides a local structural signal for distinguishing between licit and illicit behaviors.

In [ ]:
clustering = nx.clustering(G_undirected)
df_scores['clustering_coef'] = df_scores['txId'].map(clustering).fillna(0)

In [ ]:
# small clustering coefficient
def find_and_plot_cliques(
    G, class_map=None, min_k=3, max_k=7, target=None, max_plots=4, seed=42
):
    # 1) Enumerate cliques up to size max_k
    clqs = []
    for c in nx.enumerate_all_cliques(G_undirected):
        if len(c) > max_k:
            break  # enumerate_all_cliques yields in nondecreasing size
        if min_k <= len(c) <= max_k:
            clqs.append(c)

    # 2) (Optional) prioritize cliques that contain the target
    if target is not None and target in G_undirected:
        clqs_target = [c for c in clqs if target in c]
        clqs_other  = [c for c in clqs if target not in c]
        clqs = clqs_target + clqs_other  # target-first

    # 3) Sort by size (bigger first) and keep top few
    clqs.sort(key=len, reverse=True)
    pick = clqs[:max_plots]

    # 4) Quick summary of how many cliques by size
    size_counts = Counter(map(len, clqs))
    print("Cliques by size (within range):", dict(sorted(size_counts.items())))

    # 5) Plot each chosen clique as its own tiny subgraph
    for i, c in enumerate(pick, 1):
        H = G_undirected.subgraph(c).copy()

        # Node colors by label (if provided)
        if class_map:
            def col(n):
                lab = class_map.get(n)
                return {'illicit':'#d62728','licit':'#2ca02c'}.get(lab, '#888888')
            node_colors = [col(n) for n in H.nodes()]
        else:
            node_colors = '#1f77b4'

        plt.figure(figsize=(3.2, 3.2))
        pos = nx.circular_layout(H) if len(H) >= 5 else nx.spring_layout(H, seed=seed)
        nx.draw(
            H, pos,
            with_labels=True, font_size=8,
            node_color=node_colors, node_size=250,
            edge_color="#999999"
        )
        plt.title(f"Clique #{i}: K_{len(H)}")
        plt.tight_layout()
        plt.show()

# Example usage:
# find_and_plot_cliques(G_undirected, class_map=class_map, min_k=3, max_k=6, target=target, max_plots=4)

In [ ]:
find_and_plot_cliques(G_undirected, class_map=class_map, min_k=3, max_k=7, target=target, max_plots=4)

This is a complete 6-node subgraph (clique), where every node connects to every other. Interestingly, the nodes here are not exclusively illicit or licit — it’s a fully connected structure without a clear fraud signature. Such cliques can still be useful signals, since they represent unusually dense transaction patterns that may not arise randomly.

In [ ]:
def clique_stats_illicit(
    G, class_map, min_k=3, max_k=8, min_illicit=1, show_examples=3, seed=42
):
    """
    Count cliques by size and the subset that include >= min_illicit illicit nodes.
    Optionally plot a few example illicit cliques.
    """
    # Enumerate cliques up to max_k
    all_cliques = []
    illicit_cliques = []
    for c in nx.enumerate_all_cliques(G_undirected):
        k = len(c)
        if k > max_k:
            break
        if k >= min_k:
            all_cliques.append(c)
            num_illicit = sum(1 for n in c if class_map.get(n) == 'illicit')
            if num_illicit >= min_illicit:
                illicit_cliques.append(c)

    all_counts = Counter(map(len, all_cliques))
    illicit_counts = Counter(map(len, illicit_cliques))

    print("All cliques by size:", dict(sorted(all_counts.items())))
    print(f"Cliques with ≥{min_illicit} illicit nodes:", dict(sorted(illicit_counts.items())))

    # Show a few example illicit cliques, largest first
    illicit_cliques.sort(key=len, reverse=True)
    for i, c in enumerate(illicit_cliques[:show_examples], 1):
        H = G_undirected.subgraph(c).copy()
        node_colors = [
            {'illicit':'#d62728', 'licit':'#2ca02c'}.get(class_map.get(n), '#888888')
            for n in H.nodes()
        ]
        plt.figure(figsize=(3.2, 3.2))
        pos = nx.circular_layout(H) if len(H) >= 5 else nx.spring_layout(H, seed=seed)
        nx.draw(H, pos, with_labels=True, font_size=8,
                node_color=node_colors, node_size=260, edge_color="#999999")
        plt.title(f"Illicit clique example: K_{len(H)}")
        plt.tight_layout()
        plt.show()

    return all_counts, illicit_counts

# Example usage (undirected graph):
# all_counts, illicit_counts = clique_stats_illicit(
#     G_undirected, class_map, min_k=3, max_k=8, min_illicit=1, show_examples=4
# )

In [ ]:
clique_stats_illicit(G_undirected, class_map, min_k=3, max_k=8, min_illicit=1, show_examples=4)

This smaller 3-node clique consists only of illicit transactions. The tight triangular pattern is highly suspicious: it may represent repeated circular transfers, coordinated activity, or laundering of funds among a small group of actors. These tightly knit structures highlight how clique complexes can reveal hidden fraud “cells” within the broader network.

## Shortest Path to Illicit

We compute the shortest path length from each node to the nearest known illicit transaction. This is a fundamental concept in graph theory, where the shortest path is the minimum number of edges required to travel between two nodes Newman p. 139-140 [5] and West p. 97, [10]. We will use `igraph` Which is faster than `networksx`.

For fraud detection, proximity to known illicit nodes is a powerful signal: the closer a transaction is to confirmed fraud, the higher its likelihood of also being suspicious. This measure leverages the intuition that fraudulent activity tends to propagate locally within the network, forming dense clusters of illicit transactions.

In [ ]:
# Example using your edgelist DataFrame
# nx.Graph or DiGraph
G_nx = nx.from_pandas_edgelist(df_edgelist, source='source', target='target', create_using=nx.Graph())

# --- Convert NetworkX to iGraph ---
# Ensure all node IDs are strings (igraph requires this)
G_nx = nx.relabel_nodes(G_nx, lambda x: str(x))

# Create igraph from edge list
edges = list(G_nx.edges())
G_ig = ig.Graph.TupleList(edges, directed=G_nx.is_directed())

# Identify illicit nodes
illicit_nodes = [str(n) for n, label in class_map.items() if label == 'illicit']

# Prepare batch loop
all_nodes = G_ig.vs['name']
batch_size = 100

for batch_num, i in enumerate(range(0, len(all_nodes), batch_size)):
    batch = all_nodes[i:i + batch_size]

    try:
        dists_matrix = G_ig.shortest_paths(source=batch, target=illicit_nodes)
    except Exception as e:
        print(f"Error in batch {i}-{i+batch_size}: {e}")
        continue

    # Collect results
    results = []
    for j, node in enumerate(batch):
        dists = dists_matrix[j]
        min_dist = min([d for d in dists if d < float('inf')], default=None)
        results.append({'txId': node, 'shortest_path_to_illicit': min_dist})

    # Save current batch
    batch_df = pd.DataFrame(results)
    batch_df.to_csv(f"shortest_path_batch_{batch_num}.csv", index=False)

In [ ]:
files = glob.glob("shortest_path_batch_*.csv")
df_all = pd.concat([pd.read_csv(f) for f in files], ignore_index=True)

In [ ]:
# Optional: check the result
print(df_all.head())
print(df_all.shape)

In [ ]:
# Convert both to string before merging add this before the merge:
df_scores['txId'] = df_scores['txId'].astype(str)
df_all['txId'] = df_all['txId'].astype(str)

# Then safely merge:
df_scores = df_scores.merge(df_all, on='txId', how='left')

In [ ]:
def _cycle_pos(cycle, scale=(1.0, 1.0), rotation=0.0):
    """Positions nodes around an ellipse with optional rotation (radians)."""
    L = len(cycle)
    angles = np.linspace(0, 2*np.pi, L, endpoint=False) + rotation
    ax, ay = scale
    return {node: (ax*np.cos(a), ay*np.sin(a)) for node, a in zip(cycle, angles)}

def _node_color(node, class_map):
    cls = class_map.get(node, "unknown")
    return "red" if cls=="illicit" else ("green" if cls=="licit" else "gray")

def plot_one_cycle(cycle_nodes, class_map, ax, title="", scale=(1.0,1.0), rotation=0.0, node_size=60):
    cyc = list(cycle_nodes)
    H = nx.Graph()
    H.add_nodes_from(cyc)
    H.add_edges_from(zip(cyc, cyc[1:] + [cyc[0]]))  # ring
    pos = _cycle_pos(cyc, scale=scale, rotation=rotation)
    colors = [_node_color(n, class_map) for n in cyc]
    nx.draw(H, pos, ax=ax, with_labels=False, node_color=colors, edge_color="#999999", node_size=node_size)
    ax.set_title(title)
    ax.set_xticks([]); ax.set_yticks([])
    ax.set_aspect('equal')

def plot_top_k_cycles_panel(sus_cycles, class_map, k=4):
    rows = 2 if k > 2 else 1
    cols = 2 if k > 1 else 1
    fig, axes = plt.subplots(rows, cols, figsize=(10, 10) if rows==2 else (10,5))
    axes = np.array(axes).reshape(-1)  # flatten

    # choose a few shapes/rotations to “spread out” the look
    shapes = [((1.3,0.8), 0.3), ((1.3,0.8), 0.3), ((1.3,0.8), 0.3), ((1.3,0.8), 0.3)]

    for i, info in enumerate(sus_cycles[:k]):
        scale, rot = shapes[i % len(shapes)]
        title = f"Cycle #{i+1} (len={info['len']}, ill={info['illicit']}, r={info['ratio']:.2f})"
        plot_one_cycle(info["nodes"], class_map, axes[i], title=title, scale=scale, rotation=rot, node_size=50)

    # hide any unused axes
    for j in range(len(sus_cycles[:k]), len(axes)):
        axes[j].axis('off')

    plt.tight_layout()


In [ ]:
# Four most suspicious cycles as a 2x2 panel
plot_top_k_cycles_panel(sus_cycles, class_map, k=4)

Top Suspicious Cycles (uniform ellipse layout).
Each ring shows a single simple cycle from the Paton (DFS) cycle basis. We rank cycles by an illicit-presence score and plot the top four. Notice the long contiguous red segments and high illicit ratios (0.42–0.72), consistent with coordinated circular flows—classic laundering patterns. Unknown nodes embedded in these rings are strong follow-ups for investigation.

## Role Embeddings

We used role-based embeddings to capture the structural function of each node in the transaction graph. Unlike neighborhood-based embeddings (e.g., Node2Vec), which encode proximity, role embeddings map nodes with similar structural patterns (e.g., hubs, bridges, periphery) into similar vector representation. See Grover and Leskovec, *node2vec: Scalable Feature Learning for Networks* [20]

In fraud detection, this allows us to identify transactions that “behave” like known illicit hubs, even if they are far apart in the network. Role embeddings provide a generalized, position-aware feature space that complements community and centrality measures.

In [ ]:

node2vec = Node2Vec(G_undirected, dimensions=32, walk_length=30, num_walks=200, workers=4)
model = node2vec.fit(window=10, min_count=1)

# Convert to DataFrame
role_df = pd.DataFrame([model.wv[str(n)] for n in G_undirected.nodes()], columns=[f'role_{i}' for i in range(32)])
role_df['txId'] = list(G_undirected.nodes())
df_scores = df_scores.merge(role_df, on='txId', how='left')

# 7 hours


In [ ]:
# Reload your features
df_scores = pd.read_parquet('/content/drive/MyDrive/df_scores_v1.parquet')

# Extract role embedding columns
role_cols = [c for c in df_scores.columns if c.startswith("role_")]
emb = df_scores[['txId'] + role_cols].copy()

# Add class labels
emb['class'] = df_scores['label']

# Sample if huge
MAXN = 20000
if len(emb) > MAXN:
    emb = emb.sample(MAXN, random_state=42)

X = emb[role_cols].to_numpy()

# PCA projection
pca = PCA(n_components=2, random_state=42)
Z = pca.fit_transform(X)
emb['z1'], emb['z2'] = Z[:,0], Z[:,1]

# Plot
color_map = {'illicit':'red','licit':'green','unknown':'gray'}
colors = emb['class'].map(color_map)

plt.figure(figsize=(7,6))
plt.scatter(emb['z1'], emb['z2'], c=colors, s=8, alpha=0.7, linewidths=0)
plt.title('Role Embeddings (PCA 2D)')
plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}% var)')
plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}% var)')
for k,v in color_map.items():
    plt.scatter([],[],c=v,s=30,label=k)
plt.legend(frameon=False, markerscale=1.5)
plt.tight_layout()
plt.show()

In [ ]:
# Rebalance: keep all illicit, sample licit/unknown to match
emb2 = emb.copy()
n_illicit = (emb2['class']=='illicit').sum()
sampled = [
    emb2[emb2['class']=='illicit'],
    emb2[emb2['class']=='licit'  ].sample(min(n_illicit, (emb2['class']=='licit').sum()),  random_state=42),
    emb2[emb2['class']=='unknown'].sample(min(n_illicit, (emb2['class']=='unknown').sum()),random_state=42),
]
emb_bal = pd.concat(sampled).sample(frac=1, random_state=42)  # shuffle

Xb = emb_bal[role_cols].to_numpy().astype(np.float32)
Xb = StandardScaler().fit_transform(Xb)  # z-score

In [ ]:
# Try UMAP; fallback to t-SNE if not installed
use_umap = False
try:
    import umap
    use_umap = True
except Exception:
    from sklearn.manifold import TSNE

colors = emb_bal['class'].map({'illicit':'red','licit':'green','unknown':'gray'})

# PCA
pca = PCA(n_components=2, random_state=42)
Zp = pca.fit_transform(Xb)

# UMAP / t-SNE
if use_umap:
    reducer = umap.UMAP(n_components=2, n_neighbors=30, min_dist=0.1, random_state=42)
    Zu = reducer.fit_transform(Xb)
    right_title = "UMAP 2D"
else:
    Zu = TSNE(n_components=2, perplexity=30, learning_rate='auto', init='pca', random_state=42).fit_transform(Xb)
    right_title = "t-SNE 2D"

# Plot panel
fig, axes = plt.subplots(1, 2, figsize=(12,5))

axes[0].scatter(Zp[:,0], Zp[:,1], c=colors, s=8, alpha=0.7, linewidths=0)
axes[0].set_title(f"PCA 2D (PC1 {pca.explained_variance_ratio_[0]*100:.1f}%, PC2 {pca.explained_variance_ratio_[1]*100:.1f}%)")
axes[0].set_xlabel("PC1"); axes[0].set_ylabel("PC2")

axes[1].scatter(Zu[:,0], Zu[:,1], c=colors, s=8, alpha=0.7, linewidths=0)
axes[1].set_title(f"Node2Vec Embedding ({right_title})")
axes[1].set_xlabel("dim1"); axes[1].set_ylabel("dim2")

# tiny legend (once)
for k,v in {'illicit':'red','licit':'green','unknown':'gray'}.items():
    axes[0].scatter([],[],c=v,s=30,label=k)
axes[0].legend(frameon=False, markerscale=1.5, loc='upper right')

plt.tight_layout()
plt.show()

Left plot (PCA 2D)

PCA is a linear projection: it just rotates the 32-dim embedding space into the top variance directions. It’s useful for variance explanation (e.g. “PC1 explains 4.5%”). But because `Node2Vec` embeddings are spread across many dimensions, PCA usually looks like a dense cloud with no clear clusters.

Right plot (UMAP 2D)

UMAP is a nonlinear manifold learner: it tries to preserve local neighborhoods. That’s why distinct **islands** forming.

In fraud context: those islands often correspond to structurally similar transaction patterns. The fact that many red (illicit) points concentrate together means UMAP is pulling out behavioral clusters that PCA can’t.

## Spectral Partitioning

Spectral partitioning divides a graph using eigenvectors of the graph Laplacian. This method minimizes the “cut” between groups while balancing their sizes, providing a mathematically elegant way of revealing network structure, and see Newman, *Spectral methods for community detection and graph partitioning* [21].

For fraud detection, spectral methods highlight global partitions in the transaction graph. Since illicit activity often forms dense, tightly connected subgraphs, spectral clustering can help separate suspicious subnetworks from the broader flow of licit transactions.

In [ ]:
# 1. Create node index mapping for reproducibility
nodes = list(G_undirected.nodes())
node_index = {n: i for i, n in enumerate(nodes)}

# 2. Build normalized Laplacian matrix
L = nx.normalized_laplacian_matrix(G_undirected, nodelist=nodes)

# 3. Compute the first k eigenvectors (skip the trivial all-ones vector at index 0)
k = 5  # you can tune this; 5 gives you 5 global features
vals, vecs = eigsh(L, k=k, which='SM')  # smallest magnitude eigenvalues

# 4. Drop the first eigenvector (trivial), keep others
spectral_features = vecs[:, 1:]  # shape: (num_nodes, k-1)

# 5. Add to df_scores
spec_df = pd.DataFrame(
    spectral_features,
    index=nodes,
    columns=[f'spectral_ev{i}' for i in range(1, spectral_features.shape[1] + 1)]
).reset_index().rename(columns={'index': 'txId'})

df_scores = df_scores.merge(spec_df, on='txId', how='left')

# 10 min

In [ ]:
# Filter just the columns we need
plot_df = df_scores[['spectral_ev1', 'spectral_ev2', 'label']].copy()

# Map labels to colors
color_map = {'illicit': '#d62728', 'licit': '#2ca02c', 'unknown': '#888888'}
colors = plot_df['label'].map(color_map)

plt.figure(figsize=(7,6))
plt.scatter(
    plot_df['spectral_ev1'],
    plot_df['spectral_ev2'],
    c=colors,
    alpha=0.6,
    s=10,
    edgecolors='none'
)

# Legend
for lbl, col in color_map.items():
    plt.scatter([], [], c=col, label=lbl, s=30)
plt.legend(title='Label', loc='upper right')

plt.xlabel('Spectral EV1')
plt.ylabel('Spectral EV2')
plt.title('Spectral Embedding of Elliptic Graph (EV1 vs EV2)')
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Keep only needed cols and drop NaNs (just in case)
plot3 = df_scores[['spectral_ev1', 'spectral_ev2', 'spectral_ev3', 'label']].dropna()

# Map labels to colors
cmap = {'illicit': '#d62728', 'licit': '#2ca02c', 'unknown': '#888888'}
colors = plot3['label'].map(cmap)

# subsample for speed if it's too dense
max_points = 100_000
if len(plot3) > max_points:
    plot3 = plot3.sample(max_points, random_state=42)
    colors = plot3['label'].map(cmap)

fig = plt.figure(figsize=(8,7))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(
    plot3['spectral_ev1'], plot3['spectral_ev2'], plot3['spectral_ev3'],
    c=colors, s=6, depthshade=False
)

ax.set_xlabel('Spectral EV1'); ax.set_ylabel('Spectral EV2'); ax.set_zlabel('Spectral EV3')
ax.set_title('Elliptic Spectral Embedding (EV1, EV2, EV3)')

# Legends via proxy artists
for lbl, col in cmap.items():
    ax.scatter([], [], [], c=col, s=20, label=lbl)
ax.legend(loc='upper left')
plt.show()

# Act 2 Topology

## Simplicial Complexes

We extend graphs into higher-dimensional objects using simplicial complexes, where nodes, edges, triangles, and higher-order cliques form building blocks for topology. A clique complex (or flag complex) is a simplicial complex constructed from all complete subgraphs (cliques) in the network. See Ghrist, *Barcodes: The persistent topology of data* [22].

In fraud detection, higher-order connectivity can reveal patterns missed by pairwise edges. For example, tightly interlinked groups of transactions forming large cliques may correspond to collusive fraud rings. By lifting graphs into clique complexes, we capture not only who connects to whom, but also the multi-way entanglements of suspicious entities.

In [ ]:
# config
# K = 6
K = 6

# bail-out which a guardrail
FACE_CAP = 5_000_000
OUT_PREFIX = f"tda_filtr_flagK{K}_epsdim"

# save finite H1 lifetimes table per component
SAVE_DIAGS = True

# None = all
TOP_N_COMPONENTS = None

# Filtration source: node time (lower = earlier)
node_time = {str(tid): int(ts) for tid, ts in df_features[['txId', 'time_step']].itertuples(index=False)}

def vtime(u):
    """Return node's time_step or a very large value if missing."""
    return float(node_time.get(str(u), 10**9))

def face_filtration(face):
    """
    Upper-star style:
      filtration(face) = max vertex time + epsilon * dimension
    This ensures vertices < edges < triangles < tets appear strictly later,
    avoiding 'birth=death' collapses at a single time.
    """
    t = max(vtime(v) for v in face)
    dim = len(face) - 1
    return t + dim * 1e-6

def insert_flag_with_filtration(H, K, face_cap=FACE_CAP):
    """Insert vertices and all faces from cliques up to size K with filtration."""
    st = gudhi.SimplexTree()
    inserted = 0

    # vertices
    for u in H.nodes():
        st.insert([int(u)], filtration=face_filtration([u]))
        inserted += 1
        if inserted > face_cap:
            return st, inserted

    # cliques up to size K (global cutoff)
    for clique in nx.enumerate_all_cliques(H):
        if len(clique) > K:
            break
        if len(clique) >= 2:
            for r in range(2, len(clique) + 1):
                for face in combinations(clique, r):
                    st.insert(list(map(int, face)), filtration=face_filtration(face))
                    inserted += 1
                    if inserted > face_cap:
                        return st, inserted
    return st, inserted

def summarize_h1(diag):
    """Return stats for finite H1, plus counts of zero-length and essential (infinite) classes."""
    finite = []
    zero_len = 0
    essential = 0
    for dim, (b, d) in diag:
        if dim != 1:
            continue
        if np.isinf(d):
            essential += 1
        else:
            if d == b:
                zero_len += 1
            else:
                finite.append(d - b)
    if not finite:
        return {
            "h1_finite_count": 0,
            "h1_total_pers": 0.0,
            "h1_max_pers": 0.0,
            "h1_mean_pers": 0.0,
            "h1_median_pers": 0.0,
            "h1_zero_len": zero_len,
            "h1_essential": essential
        }, pd.DataFrame(columns=["birth","death","lifetime"])

    L = np.array(finite, float)
    return {
        "h1_finite_count": int(len(L)),
        "h1_total_pers": float(L.sum()),
        "h1_max_pers": float(L.max()),
        "h1_mean_pers": float(L.mean()),
        "h1_median_pers": float(np.median(L)),
        "h1_zero_len": zero_len,
        "h1_essential": essential
    }, pd.DataFrame({"birth": [], "death": [], "lifetime": []})  # keep small; write detailed diags if you want

# resume detection
done = {
    int(f.split("_")[-1].split(".")[0])
    for f in os.listdir()
    if f.startswith(OUT_PREFIX + "_summary_") and f.endswith(".csv")
}

# choose components and largest sorted
components = sorted(nx.connected_components(G_undirected), key=len, reverse=True)
if TOP_N_COMPONENTS is not None:
    components = components[:TOP_N_COMPONENTS]

print(f"[INFO] Nodes={G_undirected.number_of_nodes():,}, Edges={G_undirected.number_of_edges():,}")
print(f"[INFO] Components to process: {len(components)} (resume skips {len(done)})")

start = time.time()
processed = 0

for cid, nodes in enumerate(components):
    if cid in done:
        print(f"[SKIP] comp {cid} (resume)")
        continue

    H = G_undirected.subgraph(nodes).copy()
    n, m = H.number_of_nodes(), H.number_of_edges()
    print(f"[PROC] comp {cid}: n={n:,}, m={m:,}")

    st, faces = insert_flag_with_filtration(H, K, FACE_CAP)
    print(f"       inserted faces={faces:,}")

    st.compute_persistence(homology_coeff_field=2)
    bettis = st.betti_numbers()
    beta0 = bettis[0] if len(bettis) > 0 else 0
    beta1 = bettis[1] if len(bettis) > 1 else 0

    diag = st.persistence()
    h1_stats, h1_df = summarize_h1(diag)
    rss = psutil.Process().memory_info().rss / 1e9

    summary_path = f"{OUT_PREFIX}_summary_{cid}.csv"
    pd.DataFrame([{
        "component_id": cid,
        "n": n, "m": m,
        "betti_0": beta0, "betti_1": beta1,
        **h1_stats,
        "faces": faces, "rss_gb": rss
    }]).to_csv(summary_path, index=False)
    print(f"       wrote {summary_path} | β0={beta0}, β1={beta1}, "
          f"finiteH1={h1_stats['h1_finite_count']}, zero={h1_stats['h1_zero_len']}, inf={h1_stats['h1_essential']}, RSS={rss:.2f} GB")

    if SAVE_DIAGS and not h1_df.empty:
        h1_df.to_csv(f"{OUT_PREFIX}_h1_{cid}.csv", index=False)

    processed += 1
    del st, H, diag, h1_df
    gc.collect()

print(f"[DONE] processed {processed} components in {time.time()-start:.1f}s")

# Merge summaries
all_summ = sorted(glob.glob(f"{OUT_PREFIX}_summary_*.csv"))
if all_summ:
    merged = pd.concat((pd.read_csv(p) for p in all_summ), ignore_index=True)
    merged.to_csv(f"{OUT_PREFIX}_summary_all.csv", index=False)
    print(f"[INFO] merged -> {OUT_PREFIX}_summary_all.csv, shape={merged.shape}")

In [ ]:
# merge component summaries
summ_files = sorted(glob.glob("tda_filtr_flagK6_epsdim_summary_*.csv"))
tda_comp = pd.concat((pd.read_csv(p) for p in summ_files), ignore_index=True)

# map node -> component_id (same order used by your run: largest-first)
comp_map = {}
for cid, nodes in enumerate(sorted(nx.connected_components(G_undirected), key=len, reverse=True)):
    for n in nodes:
        comp_map[str(n)] = cid

comp_df = pd.DataFrame({"txId": list(comp_map.keys()),
                        "component_id": list(comp_map.values())})

# 1) Make sure both sides use strings for txId
df_scores["txId"] = df_scores["txId"].astype(str)
comp_df["txId"]  = comp_df["txId"].astype(str)

# 2) (Optional) sanity check
print("dtypes:", df_scores["txId"].dtype, comp_df["txId"].dtype)

# 3) Merge (node_comp is one row per txId, so many_to_one is correct)
n_before = len(df_scores)
df_scores = df_scores.merge(comp_df, on="txId", how="left", validate="many_to_one")
assert len(df_scores) == n_before



## Persistent Homology

Persistent homology tracks topological features (connected components, loops, voids) across multiple scales, forming a “barcode” of network structure Ghrist [22] and Edelsbrunner, Harer [14]. By observing which features persist as we vary thresholds (e.g., transaction amounts or connectivity), we separate noise from meaningful patterns.

In fraud detection, persistence can reveal hidden cycles or higher-order structures in transaction flows that remain stable across scales, offering robust indicators of illicit behavior even under noisy or incomplete data.

In [ ]:


# Select role embedding columns
role_cols = [col for col in df_scores.columns if col.startswith("role_")]
data = df_scores[role_cols].values

# Apply PCA to reduce dimensionality before TDA
pca = PCA(n_components=10, random_state=42)
data_pca = pca.fit_transform(data)

# Batch settings
batch_size = 500  # Adjust based on memory
num_batches = int(np.ceil(data_pca.shape[0] / batch_size))
results = []

# Directory to save results
os.makedirs("/mnt/data/tda_batches", exist_ok=True)

# Run persistent homology in batches
for i in tqdm(range(num_batches)):
    batch_data = data_pca[i * batch_size : (i + 1) * batch_size]

    try:
        rips = gudhi.RipsComplex(points=batch_data)
        simplex_tree = rips.create_simplex_tree(max_dimension=2)
        diag = simplex_tree.persistence()

        # Save the diagram to file
        with open(f"/mnt/data/tda_batches/persistence_batch_{i}.pkl", "wb") as f:
            pickle.dump(diag, f)

    except Exception as e:
        results.append((i, f"Failed: {str(e)}"))
    else:
        results.append((i, "Success"))

# 2 hr

$H_1$ measures loops. In transaction graphs, persistent $H_1$ features correspond to ring-laundering patterns—value that circulates through multiple addresses and returns—whereas $H_0$ only reflects fragmentation and $H_2$ is rarely present. Ranking by persistence suppresses noisy, low-weight loops and highlights coordinated activity. $H_0$ and $H_1$ is all we need to conduct this algorithm.

In [ ]:
# Define feature extraction function
def summarize_diagram(diag):
    h0_lifetimes = [death - birth for dim, (birth, death) in diag if dim == 0 and np.isfinite(death)]
    h1_lifetimes = [death - birth for dim, (birth, death) in diag if dim == 1 and np.isfinite(death)]

    return {
        "tda_h0_count": len(h0_lifetimes),
        "tda_h0_max": max(h0_lifetimes) if h0_lifetimes else 0,
        "tda_h0_mean": np.mean(h0_lifetimes) if h0_lifetimes else 0,
        "tda_h1_count": len(h1_lifetimes),
        "tda_h1_max": max(h1_lifetimes) if h1_lifetimes else 0,
        "tda_h1_mean": np.mean(h1_lifetimes) if h1_lifetimes else 0,
    }

# Loop through your saved .pkl files and collect features
tda_features = []

for i in range(num_batches):
    with open(f"/mnt/data/tda_batches/persistence_batch_{i}.pkl", "rb") as f:
        diag = pickle.load(f)

    summary = summarize_diagram(diag)
    summary["batch_id"] = i
    tda_features.append(summary)

df_tda = pd.DataFrame(tda_features)

# Expand the df_scores to include those batches
tda_rows = []

for i, row in df_tda.iterrows():
    start_idx = i * batch_size
    end_idx = min((i + 1) * batch_size, len(df_scores))

    for _ in range(start_idx, end_idx):
        tda_rows.append(row.drop("batch_id").to_dict())

df_tda_expanded = pd.DataFrame(tda_rows)

# Now merge
df_scores = pd.concat([df_scores.reset_index(drop=True), df_tda_expanded.reset_index(drop=True)], axis=1)

plot


In [ ]:
def compute_ph_and_plot(st, title_prefix="Persistent Homology"):
    # Full persistence
    pers = st.persistence(homology_coeff_field=2, persistence_dim_max=True)

    # Split by dimension
    H0 = [(b, d) for dim, (b, d) in pers if dim == 0]
    H1 = [(b, d) for dim, (b, d) in pers if dim == 1]

    def lifetimes(pairs):
        L, zero_len, essential = [], 0, 0
        for b, d in pairs:
            if d == float('inf'):
                essential += 1
            else:
                life = d - b
                if life <= 0: zero_len += 1
                L.append(max(0.0, life))
        return np.asarray(L, float), zero_len, essential

    L1, z1, e1 = lifetimes(H1)

    # --- Barcode plots (H0/H1)
    def barcode(ax, pairs, title):
        if not pairs:
            ax.set_title(title + " (empty)")
            ax.axis('off')
            return
        y = 0
        for (b, d) in pairs:
            d = (10 if d == float('inf') else d)  # cap infinities for display
            ax.hlines(y, b, d, linewidth=2)
            y += 1
        ax.set_title(title)
        ax.set_xlabel("filtration")
        ax.set_ylabel("bars")
        ax.grid(alpha=0.2)

    fig = plt.figure(figsize=(10,5))
    ax1 = fig.add_subplot(1,2,1); barcode(ax1, H0, f"{title_prefix}: H0 barcode")
    ax2 = fig.add_subplot(1,2,2); barcode(ax2, H1, f"{title_prefix}: H1 barcode")
    plt.tight_layout(); plt.show()

    # --- H1 persistence diagram (finite only)
    finite = np.array([[b, d] for (b, d) in H1 if d != float('inf')], float)
    if finite.size > 0:
        plt.figure(figsize=(5,5))
        plt.scatter(finite[:,0], finite[:,1], s=16)
        maxv = max(finite.max(), 1.0)
        plt.plot([0, maxv], [0, maxv], ls='--', lw=1)
        plt.xlabel("birth"); plt.ylabel("death")
        plt.title(f"{title_prefix}: H1 persistence diagram")
        plt.grid(alpha=0.2); plt.show()
    else:
        print("Note: no finite H1 bars (all loops are essential or H1 is empty). Skipping H1 PD.")

    stats = dict(
        h1_finite_count=int(len(L1)),
        h1_total_pers=float(L1.sum()) if len(L1) else 0.0,
        h1_max_pers=float(L1.max()) if len(L1) else 0.0,
        h1_mean_pers=float(L1.mean()) if len(L1) else 0.0,
        h1_median_pers=float(np.median(L1)) if len(L1) else 0.0,
        h1_zero_len=int(z1),
        h1_essential=int(e1),
    )
    return {"H0_pairs": H0, "H1_pairs": H1, "H1_stats": stats}

In [ ]:
# Build ST from a manageable subgraph H (undirected)
st = build_flag_simplextree_up_to_2(H)
ph = compute_ph_and_plot(st, title_prefix="Community PH")
print(ph["H1_stats"])

We applied persistent homology to the clique (flag) complex built from each transaction subgraph. The barcode below summarizes the evolution of topological features across the filtration.

$H_0$ (connected components): Each short bar represents a connected component at the beginning; these rapidly merge, leaving a **single dominant component** that persists.

$H_1$ (loops): We observe one long bar, corresponding to a loop that **never gets filled** in by higher-order simplices. In the fraud detection setting, such stable loops may correspond to ring-like transaction structures that are robust across scales.

Persistence diagrams and barcodes provide a complementary view to Betti numbers: instead of just a snapshot count, they reveal which topological patterns endure and which **vanish quickly** (noise).

plot


In [ ]:
def plot_persistence_diagram(st, title_prefix="Persistence Diagram"):
    pers = st.persistence(homology_coeff_field=2, persistence_dim_max=True)

    # Extract H1 finite pairs
    H1 = [(b, d) for dim, (b, d) in pers if dim == 1 and d != float("inf")]

    if not H1:
        print("No finite H1 pairs — only essential loops or none at all.")
        return

    B = np.array(H1, float)

    plt.figure(figsize=(5,5))
    plt.scatter(B[:,0], B[:,1], s=20, c="blue", alpha=0.7, label="H1 features")

    # diagonal reference line
    maxv = max(B.max(), 1.0)
    plt.plot([0, maxv], [0, maxv], ls="--", lw=1, color="black")

    plt.xlabel("Birth")
    plt.ylabel("Death")
    plt.title(f"{title_prefix}: H1 Persistence Diagram")
    plt.legend()
    plt.grid(alpha=0.3)
    plt.show()

In [ ]:
st = build_flag_simplextree_up_to_2(H)
plot_persistence_diagram(st, title_prefix="Community PH")

This subgraph contains a loop that persists all the way through — it is never filled by higher-order simplices in the flag complex. In fraud detection terms, that’s a stable ring structure, not just a temporary cycle.

In [ ]:
def plot_persistence_diagram_with_infinity(st, cap=10, title_prefix="Persistence Diagram"):
    pers = st.persistence(homology_coeff_field=2, persistence_dim_max=True)
    H1 = [(b, d) for dim, (b, d) in pers if dim == 1]

    if not H1:
        print("No H1 features at all.")
        return

    B = []
    for (b,d) in H1:
        if d == float('inf'):
            d = cap  # cap infinite bars for display
        B.append((b,d))

    B = np.array(B, float)

    plt.figure(figsize=(5,5))
    plt.scatter(B[:,0], B[:,1], s=20, c="blue", alpha=0.7)
    plt.plot([0, cap], [0, cap], ls="--", lw=1, color="black")
    plt.xlabel("Birth")
    plt.ylabel("Death (∞ capped)")
    plt.title(f"{title_prefix}: H1 Persistence Diagram (∞ capped at {cap})")
    plt.grid(alpha=0.3)
    plt.show()

In [ ]:
plot_persistence_diagram_with_infinity(st, cap=10, title_prefix="Community PH")

In the persistence diagram, the single blue point corresponds to a 1-dimensional loop that appears early in the filtration and persists indefinitely $(\text{death} =  \infty$, here capped at 10 for visualization). The dashed diagonal marks $\text{birth} = \text{death}$; features near the diagonal are noise, while features far above are robust. This result indicates that the community contains a stable cycle structure that is never filled in by higher-order simplices, consistent with a ring-like pattern of transactions.

## Clique Complex (flag complex) from graph

Simplicial & Clique (Flag) Complexes. We lift the transaction graph into a simplicial complex to capture higher-order structure (edges, triangles, and higher cliques). In particular, we use the clique (flag) complex, which adds a $k$-simplex for every $(k+1)$-clique in the graph; this is the maximal simplicial complex whose 1-skeleton equals the original network. We analyze two complementary signals. First, Betti numbers $(\beta_0, \beta_1)$ summarize the topology at a snapshot (connected components and independent loops). Second, we compute persistent homology on $H_1$ under a chosen filtration (thresholding edges/weights over scale), and summarize the barcode by the number of finite bars, total and maximum persistence, and related statistics. Betti highlights instantaneous topology; persistence highlights stability of loops across scales—useful for distinguishing robust ring-like fraud patterns from transient noise.

See Edelsbrunner and Harer [14].

In [ ]:
# Ensure nodes are integers
G_int = nx.convert_node_labels_to_integers(G_undirected)
cliques = [list(c) for c in nx.find_cliques(G_int)]

# Build the simplex tree from clique complex
simplex_tree = gudhi.SimplexTree()

for clique in cliques:
    for i in range(1, len(clique) + 1):
        for subset in combinations(clique, i):
            simplex_tree.insert(list(subset))  # insert expects a list of ints

In [ ]:
# Make sure all IDs are strings for consistency
df_edgelist = df_edgelist.astype(str)
df_features['txId'] = df_features['txId'].astype(str)
df_classes['txId'] = df_classes['txId'].astype(str)

# Build the full undirected graph
G_full = nx.from_pandas_edgelist(df_edgelist, source='source', target='target', create_using=nx.Graph())

# Group by time step
dfs_by_time = {t: df for t, df in df_features.groupby("time_step")}
time_steps = sorted(dfs_by_time.keys())

# Output directory
output_dir = "/mnt/data/clique_tda_batches"
os.makedirs(output_dir, exist_ok=True)

results = []

# Loop through time steps and build clique complexes
for t in time_steps:
    df_t = dfs_by_time[t]
    tx_ids = df_t['txId'].tolist()
    subG = G_full.subgraph(tx_ids)

    # Convert node labels to integers for GUDHI
    mapping = {node: i for i, node in enumerate(subG.nodes())}
    G_int = nx.relabel_nodes(subG, mapping)
    reverse_mapping = {v: k for k, v in mapping.items()}

    cliques = [list(c) for c in nx.find_cliques(G_int)]
    simplex_tree = gudhi.SimplexTree()

    for clique in cliques:
        for i in range(1, len(clique) + 1):
            for subset in combinations(clique, i):
                simplex_tree.insert(list(subset))

    diag = simplex_tree.persistence()
    betti_nums = simplex_tree.betti_numbers()

    h1_lifetimes = [d[1][1] - d[1][0] for d in diag if d[0] == 1 and d[1][1] < float('inf')]

    summary = {
        "time_step": t,
        "clique_betti_0": betti_nums[0] if len(betti_nums) > 0 else 0,
        "clique_betti_1": betti_nums[1] if len(betti_nums) > 1 else 0,
        "clique_total_persistence_h1": sum(h1_lifetimes),
        "clique_max_h1_lifetime": max(h1_lifetimes) if h1_lifetimes else 0,
        "clique_num_long_h1_bars": sum(1 for l in h1_lifetimes if l > 1.0)
    }

    # Save this result to file
    with open(os.path.join(output_dir, f"clique_tda_timestep_{t}.pkl"), "wb") as f:
        pickle.dump(summary, f)

    results.append(summary)

df_clique_tda = pd.DataFrame(results)

In [ ]:


# Make sure txId and time_step columns are available
df_mapping = df_features[['txId', 'time_step']].drop_duplicates()

# Merge the TDA results into that mapping
df_mapping = df_mapping.merge(df_clique_tda, on="time_step", how="left")

def merge_with_tda(df_scores, df_clique_tda):
    df_mapping = df_features[['txId', 'time_step']].drop_duplicates()
    df_mapping = df_mapping.merge(df_clique_tda, on="time_step", how="left")

    df_scores['txId'] = df_scores['txId'].astype(str)
    df_mapping['txId'] = df_mapping['txId'].astype(str)

    df_scores = df_scores.merge(df_mapping, on="txId", how="left")

    # Drop rows where TDA features are missing
    tda_cols = [c for c in df_clique_tda.columns if c != 'time_step']
    df_scores = df_scores.dropna(subset=tda_cols)

    return df_scores.reset_index(drop=True)

# Convert txId to string in both DataFrames
df_scores['txId'] = df_scores['txId'].astype(str)
df_mapping['txId'] = df_mapping['txId'].astype(str)

# Step 3: Now merge back into df_scores using txId
df_scores = df_scores.merge(df_mapping, on="txId", how="left")

In [ ]:
SAVE_DIR = "/content/drive/MyDrive/elliptic_tda"
os.makedirs(SAVE_DIR, exist_ok=True)

# --- PCA + kNN ---
role_cols = [c for c in df_scores.columns if c.startswith("role_")]
X = df_scores[role_cols].values

pca = PCA(n_components=10, random_state=42)
X_pca = pca.fit_transform(X)

nbrs = NearestNeighbors(n_neighbors=40).fit(X_pca)
_, idxs = nbrs.kneighbors(X_pca)

def summarize_H1(coords, thresh=1e-4):
    out = ripser(coords, maxdim=1)
    dgm1 = out["dgms"][1]
    if dgm1.size == 0:
        return 0.0, 0.0, 0
    lifetimes = np.maximum(dgm1[:, 1] - dgm1[:, 0], 0)
    return float(lifetimes.sum()), float(lifetimes.max(initial=0)), int((lifetimes > thresh).sum())

# --- resume state ---
resume_path = os.path.join(SAVE_DIR, "tda_resume.json")
start = 0
if os.path.exists(resume_path):
    with open(resume_path, "r") as f:
        start = json.load(f).get("start", 0)

rows = []
batch_size = 2000
N = X_pca.shape[0]
curr_start = start  # moving window start

# Ensure df_scores txId is string before any slicing
df_scores["txId"] = df_scores["txId"].astype(str)

for i in range(start, N):
    coords = X_pca[idxs[i]]
    rows.append(summarize_H1(coords))

    # flush a batch or the tail
    if (i + 1) % batch_size == 0 or i == N - 1:
        part = pd.DataFrame(
            rows,
            columns=["vr_total_persistence_h1", "vr_max_h1_lifetime", "vr_num_long_h1_bars"]
        )
        # align txIds for ONLY this batch; force str before saving
        part["txId"] = df_scores.iloc[curr_start : i + 1]["txId"].astype(str).values

        out_path = os.path.join(SAVE_DIR, f"tda_part_{curr_start}_{i}.parquet")
        part.to_parquet(out_path, index=False)

        # persist resume point and advance
        with open(resume_path, "w") as f:
            json.dump({"start": i + 1}, f)

        curr_start = i + 1
        rows = []  # reset

# --- merge all parts ---
parts = []
for f in sorted(os.listdir(SAVE_DIR)):
    if f.startswith("tda_part_") and f.endswith(".parquet"):
        p = pd.read_parquet(os.path.join(SAVE_DIR, f))
        p["txId"] = p["txId"].astype(str)
        parts.append(p)

if len(parts) == 0:
    raise ValueError("No tda_part_*.parquet files found in SAVE_DIR")

tda_all = pd.concat(parts, ignore_index=True).drop_duplicates(subset=["txId"])

# left-join back to df_scores (both sides are str)
df_scores = df_scores.merge(tda_all, on="txId", how="left")

In [ ]:
print(df_scores[["vr_total_persistence_h1","vr_max_h1_lifetime","vr_num_long_h1_bars"]].describe())
print("Zero-only rows:", (df_scores["vr_total_persistence_h1"]==0).mean())

# Save



In [ ]:
# download
# df_scores.to_parquet("/content/drive/MyDrive/df_scores_v1.parquet", index=False)

# upload
df_scores = pd.read_parquet('/content/drive/MyDrive/df_scores_v1.parquet')

In [ ]:
df_scores.info()

We engineered a comprehensive feature set for each transaction, consisting of the label, `txId`, and 61 derived features across multiple domains:

- Graph theory: degree, centralities (degree, eigenvector, betweenness), clustering coefficient, cycle scores, Louvain clustering, and spectral embeddings.

- Embeddings: role embeddings and Node2Vec (32 dimensions).

- Spectral graph features: first four eigenvectors of the graph Laplacian.

- Topological Data Analysis (TDA): persistence counts, maxima, and means for $H_0$ and $H_1$; clique Betti numbers; and persistence statistics.

- Temporal information: transaction time step.

With feature engineering complete, the next steps are to save the dataset, run sanity checks, and proceed to modeling.

## Sanity Check

In [ ]:

# 1. Check for missing values
missing = df_scores.isnull().sum()
print("\n Missing values:")
print(missing[missing > 0].sort_values(ascending=False))

# 2. Check for all-zero columns
zero_cols = df_scores.drop(columns=['txId', 'label']).columns[(df_scores.drop(columns=['txId', 'label']) == 0).all()]
print(f"\ All-zero columns: {list(zero_cols)}")

# 3. Check for low-variance columns
low_variance = df_scores.drop(columns=['txId', 'label']).std().sort_values()
print("\n Low-variance columns (bottom 5):")
print(low_variance.head(10))

# 4. Check for duplicate columns
def find_duplicate_columns(df):
    duplicate_cols = []
    cols = df.columns
    for i in range(len(cols)):
        for j in range(i+1, len(cols)):
            if df[cols[i]].equals(df[cols[j]]):
                duplicate_cols.append((cols[i], cols[j]))
    return duplicate_cols

dups = find_duplicate_columns(df_scores.drop(columns=['txId', 'label']))
print("\n Duplicate columns (exact duplicates):")
for pair in dups:
    print(pair)

# 5. Check correlation to remove highly redundant features
corr_matrix = df_scores.drop(columns=['txId', 'label']).corr().abs()
upper_triangle = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
high_corr_pairs = [(col, row) for col in upper_triangle.columns for row in upper_triangle.index if upper_triangle.loc[row, col] > 0.95]

print("\n Highly correlated feature pairs (correlation > 0.95):")
for pair in high_corr_pairs:
    print(pair)


So because the `clique_betti_0` and `pagerank` are or low variance we're going to drop those because it won't capture the information.

In [ ]:
# drop clique_betti_0 and pagerank
df_scores = df_scores.drop(columns=["clique_betti_0", "pagerank"])

Here is a correlation matrix

In [ ]:
# Compute correlation matrix
num = df_scores.drop(columns=["txId"]).select_dtypes(include=["number"])
corr = num.corr()

# Extract highly correlated pairs
threshold = 0.75
high_corr = (
    corr.where(~np.triu(np.ones(corr.shape, dtype=bool)))  # keep lower triangle only
        .stack()
        .reset_index()
)
high_corr.columns = ["feature1", "feature2", "correlation"]

# Filter above threshold
high_corr = high_corr[high_corr["correlation"].abs() > threshold] \
                     .sort_values("correlation", ascending=False)

print(high_corr)

So `deg_centrality` and `degree` are exactly the same So let's drop `deg_centrality`.

In [ ]:
# drop deg_centrality
df_scores = df_scores.drop(columns=["deg_centrality"])

In [ ]:
df_scores.describe().T

### Histograms

In [ ]:
for col in df_scores.drop(columns=["txId", "label"]).columns:
    plt.figure(figsize=(6,4))
    sns.histplot(data=df_scores, x=col, hue="label", bins=100, element="step", stat="density") #kde=True ?
    plt.title(col)
    plt.show()

### Box Plots

In [ ]:
# box plots
for col in df_scores.drop(columns=["txId", "label"]).columns:
    plt.figure(figsize=(6,4))
    sns.boxplot(data=df_scores, x=col, hue="label")
    plt.title(col)
    plt.show()

### Pair Plots

In [ ]:
sample = df_scores.sample(2000, random_state=42)  # don’t plot all 200k
sns.pairplot(sample[["degree", "betweenness", "neighbors_illicit_ratio", "tda_h1_max", "label"]],
             hue="label", corner=True, plot_kws={"alpha":0.4})
plt.show()

### Correlation Heatmap

In [ ]:
num = df_scores.drop(columns=["txId"]).select_dtypes(include=["number"])
corr = num.corr()
mask = np.triu(np.ones_like(corr, dtype=bool))

plt.figure(figsize=(12,10))
sns.heatmap(corr, mask=mask, cmap="coolwarm", center=0, vmin=-1, vmax=1)
plt.title("Feature Correlation Heatmap (upper triangle)")
plt.show()

One more time with this sanity check

In [ ]:
# 1. Check for missing values
missing = df_scores.isnull().sum()
print("\n Missing values:")
print(missing[missing > 0].sort_values(ascending=False))

# 2. Check for all-zero columns
zero_cols = df_scores.drop(columns=['txId', 'label']).columns[(df_scores.drop(columns=['txId', 'label']) == 0).all()]
print(f"\ All-zero columns: {list(zero_cols)}")

# 3. Check for low-variance columns
low_variance = df_scores.drop(columns=['txId', 'label']).std().sort_values()
print("\n Low-variance columns (bottom 5):")
print(low_variance.head(10))

# 4. Check for duplicate columns
def find_duplicate_columns(df):
    duplicate_cols = []
    cols = df.columns
    for i in range(len(cols)):
        for j in range(i+1, len(cols)):
            if df[cols[i]].equals(df[cols[j]]):
                duplicate_cols.append((cols[i], cols[j]))
    return duplicate_cols

dups = find_duplicate_columns(df_scores.drop(columns=['txId', 'label']))
print("\n Duplicate columns (exact duplicates):")
for pair in dups:
    print(pair)

# 5. Check correlation to remove highly redundant features
corr_matrix = df_scores.drop(columns=['txId', 'label']).corr().abs()
upper_triangle = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
high_corr_pairs = [(col, row) for col in upper_triangle.columns for row in upper_triangle.index if upper_triangle.loc[row, col] > 0.95]

print("\n Highly correlated feature pairs (correlation > 0.95):")
for pair in high_corr_pairs:
    print(pair)


In [ ]:
# Compute correlation matrix
num = df_scores.drop(columns=["txId"]).select_dtypes(include=["number"])
corr = num.corr()

# Extract highly correlated pairs
threshold = 0.75
high_corr = (
    corr.where(~np.triu(np.ones(corr.shape, dtype=bool)))  # keep lower triangle only
        .stack()
        .reset_index()
)
high_corr.columns = ["feature1", "feature2", "correlation"]

# Filter above threshold
high_corr = high_corr[high_corr["correlation"].abs() > threshold] \
                     .sort_values("correlation", ascending=False)

print(high_corr)

So `time_step` and `louvain_cluster` might be dropped with different modeling.

`tda_h1_count` and `tda_h0_mean`

and

`clique_betti_1` and `component_id`

Or might be drop so I'm going to keep those in mind just in case

In [ ]:
df_scores.info()

`time_step` vs `louvain_cluster`

`time_step` is chronological ordering Or a chain block of two weeks and we can see that on the visual from `df_feaures`. On the other hand, `louvain_cluster` is purely graph structural grouping dense communities.

So, they aren’t redundant, and measure different phenomena of temporal(`time_step`) and  community(`lounvain_cluster`). It is correlated, but I and Going to keep both, because models might capture interaction between time and structure.

`tda_h1_count` vs `tda_h0_mean`

`tda_h1_count` how many 1D loops/cycles persisted, while `tda_h0_mean` have a average lifetime of connected components (0D). Even if numerically correlated, they encode different topology (loops vs. connectivity). I am going to keep both, But putting aware though because it could in the long run have redundancy showing up in feature importance.

`clique_betti_1` vs `component_id`

`clique_betti_1` is Betti number to topological invariant of rank of 1st homology group. On the other hand, `component_id` is just an index for which connected component the node belongs to. These are very different in meaning. If they’re correlated, it’s dataset-specific and maybe most components are trivial.

`component_id` is actually categorical of ID label. Using it as a number can be misleading, so I’d either drop it or encode it properly like a one-hot or embeddings.

# Act III Modeling

## Feature Engineering Validation

Let's test only illict and lict, So that the feature engineerings are more robust. Let's pass other models and make sure it's accurate.

### Logistic Regression (Only on illicit and licit)

Logistic Regression expects numeric output, and fraud is often framed as:

- Illicit (fraud) = 1
- Licit (not fraud) = 0
- Unknown = drop or handle separately

Let's do a scalar score so it's on the same page with the features but not with the random forset.


In [ ]:
# Map labels
label_map = {'licit': 0, 'illicit': 1}

# Drop unknowns (optional — depends on use case)
df_logreg = df_scores[df_scores['label'].isin(label_map.keys())].copy()

# Apply mapping
df_logreg['label'] = df_logreg['label'].map(label_map)

# features
X = df_logreg.drop(columns=['txId', 'label'])

# target
y = df_logreg['label']

# Scalar standard so no bias numbers
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, stratify=y, random_state=42
)

# Fit model
logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_train, y_train)

# Predict
y_pred = logreg.predict(X_test)

# Evaluate
print(classification_report(y_test, y_pred))

# feature importance
feature_importance = pd.Series(logreg.coef_[0], index=X.columns)
top_features = feature_importance.abs().sort_values(ascending=False)

print("\n Top Features:")
print(top_features.head(10))

Looking good very accurately but of course with `'shortest_path_to_illicit'` is a huge feartue score with one feature you might need to trim that down.

In [ ]:
# Compute confusion matrix
cm = confusion_matrix(y_test, y_pred, labels=[0, 1])

# Plot
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Licit (0)', 'Illicit (1)'])
disp.plot(cmap='Blues', values_format='d')
plt.title("Confusion Matrix")
plt.show()

That's good no errors and the feature engineering are doing the job right

## Logistic Regression: illicit, licit and unknowns

Raw: The shortest path to elicit is a high feature so I'm going to keep that in mind but I just want to run it and see.

In [ ]:
def run_logistic(df_scores, mode="raw", top_n=50):
    # --- Features / target ---
    X = df_scores.drop(columns=['txId', 'label']).copy()
    y = df_scores['label'].map({'licit': 0, 'illicit': 1})

    # Transform shortest-path per mode
    if "shortest_path_to_illicit" in X.columns:
        if mode == "cap":
            X["shortest_path_to_illicit"] = X["shortest_path_to_illicit"].clip(upper=5)
        elif mode == "sigmoid":
            X["shortest_path_to_illicit"] = 1 / (1 + np.exp(X["shortest_path_to_illicit"]))
        elif mode == "drop":
            X = X.drop(columns=["shortest_path_to_illicit"])

    # Known vs unknown
    X_known, y_known = X[y.notnull()], y[y.notnull()]
    X_unknown = X[y.isnull()]

    # Train/test split on knowns
    X_train, X_test, y_train, y_test = train_test_split(
        X_known, y_known, test_size=0.3, random_state=42, stratify=y_known
    )

    # Scale (fit on train, apply to test + unknown)
    scaler = StandardScaler()
    X_train_s = scaler.fit_transform(X_train)
    X_test_s  = scaler.transform(X_test)
    X_unknown_s = scaler.transform(X_unknown)

    # Fit logistic regression
    log_reg = LogisticRegression(max_iter=1000, solver='liblinear')
    log_reg.fit(X_train_s, y_train)

    # Evaluate
    y_pred  = log_reg.predict(X_test_s)
    y_proba = log_reg.predict_proba(X_test_s)[:, 1]
    auc = roc_auc_score(y_test, y_proba)

    print(f"\n=== Logistic Mode: {mode.upper()} ===")
    print(classification_report(y_test, y_pred, digits=3))
    print("AUC:", auc)

    # Confusion matrix
    plt.figure()
    ConfusionMatrixDisplay.from_estimator(log_reg, X_test_s, y_test, cmap="Blues")
    plt.title(f"Confusion Matrix ({mode})")
    plt.show()

    # ROC curve
    fpr, tpr, _ = roc_curve(y_test, y_proba)
    plt.figure()
    plt.plot(fpr, tpr, label=f"{mode} (AUC={auc:.3f})")
    plt.plot([0,1],[0,1],"--",color="gray")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("ROC Curve")
    plt.legend()
    plt.show()

    # "Importance" for logistic = absolute standardized coefficients
    coef = pd.Series(log_reg.coef_[0], index=X_train.columns)
    coef_abs = coef.abs().sort_values(ascending=False).head(15)
    plt.figure(figsize=(8,5))
    sns.barplot(x=coef_abs.values, y=coef_abs.index)
    plt.title(f"Top |Coefficients| ({mode})")
    plt.xlabel("|standardized coefficient|")
    plt.ylabel("feature")
    plt.show()

    # Predict unknowns + save
    unknown_proba = log_reg.predict_proba(X_unknown_s)[:, 1]
    df_unknown = df_scores.loc[y.isnull(), ['txId']].copy()
    df_unknown['prob_illicit'] = unknown_proba
    df_unknown = df_unknown.sort_values('prob_illicit', ascending=False)

    out_path = f"suspicious_unknowns_log_{mode}.csv"
    df_unknown.to_csv(out_path, index=False)
    print(f"Saved top suspicious unknowns to {out_path}")

    # Show top-N preview
    display(df_unknown.head(top_n))

    return df_unknown, auc, log_reg, scaler

In [ ]:
df_log_raw, auc_raw, _, _  = run_logistic(df_scores, mode="raw", top_n=50)
df_log_cap, auc_cap, _, _  = run_logistic(df_scores, mode="cap", top_n=50)
df_log_drop, auc_drop, _, _ = run_logistic(df_scores, mode="drop", top_n=50)

### Random Forest (Only on illicit and licit)

In [ ]:
# Known labeled data
df_known = df_scores[df_scores['label'].isin(['licit', 'illicit'])].copy()
df_known['label'] = df_known['label'].map({'licit': 0, 'illicit': 1})

# Drop leakage
X_rf = df_known.drop(columns=['txId', 'label'])
y_rf = df_known['label']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X_rf, y_rf, test_size=0.2, stratify=y_rf, random_state=42
)

rf = RandomForestClassifier(
    n_estimators=100,
    class_weight='balanced',  # helpful for imbalance
    random_state=42,
    n_jobs=-1
)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)

print("\n Classification Report:")
print(classification_report(y_test, y_pred))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred, labels=[0, 1])
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Licit', 'Illicit'])
disp.plot(cmap='Blues')
plt.title("Confusion Matrix (Random Forest on Known)")
plt.show()

Looking good no errors

In [ ]:
importances = pd.Series(rf.feature_importances_, index=X_rf.columns)
top_features = importances.sort_values(ascending=False).head(15)

print("\n Top Features:")
print(top_features)

# Optional: plot
top_features.plot(kind='barh', figsize=(8, 6), title="Top Random Forest Features")
plt.gca().invert_yaxis()
plt.show()

Good again except `'shortest_path_to_illicit'` Is a huge feature block the other ones and that was the whole point to have a lot of robust features so it's more we definitely need to do something about that

## Random Forset on unknows, licit and illicit.

In [ ]:

def run_random_forest(df_scores, mode="cap", top_n=50):
    """
    Train Random Forest on licit vs illicit and apply to unknowns.
    mode: "raw", "cap", "drop"
    """
    # --- Prepare X, y ---
    X = df_scores.drop(columns=['txId', 'label']).copy()
    y = df_scores['label'].map({'licit':0, 'illicit':1})

    # Transform shortest path depending on mode
    if "shortest_path_to_illicit" in X.columns:
        if mode == "cap":
            X["shortest_path_to_illicit"] = X["shortest_path_to_illicit"].clip(upper=5)
        elif mode == "drop":
            X = X.drop(columns=["shortest_path_to_illicit"])
        # mode == "raw" → leave unchanged

    # Split into known and unknown
    X_known, y_known = X[y.notnull()], y[y.notnull()]
    X_unknown = X[y.isnull()]

    X_train, X_test, y_train, y_test = train_test_split(
        X_known, y_known, test_size=0.3, random_state=42, stratify=y_known
    )

    # --- Fit Random Forest ---
    rf = RandomForestClassifier(
        n_estimators=200,
        max_depth=None,
        class_weight="balanced",  # helps with class imbalance
        random_state=42,
        n_jobs=-1
    )
    rf.fit(X_train, y_train)

    # --- Evaluate on test ---
    y_pred = rf.predict(X_test)
    y_proba = rf.predict_proba(X_test)[:,1]

    print(f"\n=== Random Forest Mode: {mode.upper()} ===")
    print(classification_report(y_test, y_pred, digits=3))
    print("AUC:", roc_auc_score(y_test, y_proba))

    # Confusion matrix
    ConfusionMatrixDisplay.from_estimator(rf, X_test, y_test, cmap="Blues")
    plt.title(f"Confusion Matrix ({mode})")
    plt.show()

    # ROC curve
    fpr, tpr, _ = roc_curve(y_test, y_proba)
    plt.plot(fpr, tpr, label=f"{mode} (AUC={roc_auc_score(y_test,y_proba):.3f})")
    plt.plot([0,1],[0,1],"--",color="gray")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("ROC Curve")
    plt.legend()
    plt.show()

    # --- Feature importance ---
    importances = pd.Series(rf.feature_importances_, index=X_train.columns)
    top_feats = importances.sort_values(ascending=False).head(15)
    plt.figure(figsize=(8,5))
    sns.barplot(x=top_feats.values, y=top_feats.index)
    plt.title(f"Top Feature Importances ({mode})")
    plt.show()

    # --- Predict on unknowns ---
    unknown_proba = rf.predict_proba(X_unknown)[:,1]
    df_unknown = df_scores.loc[y.isnull(), ['txId']].copy()
    df_unknown['prob_illicit'] = unknown_proba
    df_unknown = df_unknown.sort_values("prob_illicit", ascending=False)

    # Save suspicious unknowns
    filename = f"suspicious_unknowns_rf_{mode}.csv"
    df_unknown.to_csv(filename, index=False)
    print(f"Saved top suspicious unknowns to {filename}")

    # Show top results
    print("\nTop suspicious unknowns:")
    display(df_unknown.head(top_n))

    return df_unknown, rf

In [ ]:
df_rf_raw, model_raw = run_random_forest(df_scores, mode="raw")
df_rf_cap, model_cap = run_random_forest(df_scores, mode="cap")
df_rf_drop, model_drop = run_random_forest(df_scores, mode="drop")

### Neural Network Classifier (Only on illicit and licit)

In [ ]:
# 1. Prepare data (licit + illicit only)
df_known = df_scores[df_scores['label'].isin(['licit', 'illicit'])].copy()
df_known['label'] = df_known['label'].map({'licit': 0, 'illicit': 1})

X_known = df_known.drop(columns=['txId', 'label'])
y_known = df_known['label']

# 2. Scale
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_known)

# 3. Split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_known, test_size=0.2, stratify=y_known, random_state=42
)

# 4. Build model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')  # Binary classification
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# 5. Train
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=64,
    callbacks=[early_stop],
    verbose=1
)

# 6. Evaluate
y_pred_probs = model.predict(X_test).flatten()
y_pred = (y_pred_probs >= 0.5).astype(int)

print(classification_report(y_test, y_pred))

# 7. Confusion matrix
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Licit", "Illicit"])
disp.plot(cmap='Blues')
plt.title("Confusion Matrix (Neural Network)")
plt.show()

Do have errors here but overall still pretty good.


## Neural Network unknows, licit and illict.

In [ ]:

def run_neural_net(df_scores, mode="raw", top_n=20):
    # --- Features/labels ---
    X = df_scores.drop(columns=['txId', 'label']).copy()
    y = df_scores['label'].map({'licit':0, 'illicit':1})

    # --- Handle shortest_path_to_illicit ---
    if "shortest_path_to_illicit" in X.columns:
        if mode == "cap":
            X["shortest_path_to_illicit"] = X["shortest_path_to_illicit"].clip(upper=5)
        elif mode == "sigmoid":
            X["shortest_path_to_illicit"] = 1 / (1 + np.exp(X["shortest_path_to_illicit"]))
        elif mode == "drop":
            X = X.drop(columns=["shortest_path_to_illicit"])

    # --- Split known/unknown ---
    X_known, y_known = X[y.notnull()], y[y.notnull()]
    X_unknown = X[y.isnull()]

    X_train, X_test, y_train, y_test = train_test_split(
        X_known, y_known, test_size=0.3, random_state=42, stratify=y_known
    )

    # --- Scale ---
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    X_unknown_scaled = scaler.transform(X_unknown)

    # --- Fit Neural Net ---
    mlp = MLPClassifier(
        hidden_layer_sizes=(64, 32),   # two hidden layers
        activation='relu',
        solver='adam',
        alpha=1e-4,
        max_iter=200,
        random_state=42
    )
    mlp.fit(X_train_scaled, y_train)

    # --- Evaluate ---
    y_pred = mlp.predict(X_test_scaled)
    y_proba = mlp.predict_proba(X_test_scaled)[:,1]
    auc = roc_auc_score(y_test, y_proba)

    print(f"\n=== Neural Net, Mode: {mode.upper()} ===")
    print(classification_report(y_test, y_pred, digits=3))
    print("AUC:", auc)

    # Confusion matrix
    ConfusionMatrixDisplay.from_estimator(mlp, X_test_scaled, y_test, cmap="Blues")
    plt.title(f"Confusion Matrix (NN, {mode})")
    plt.show()

    # ROC curve
    fpr, tpr, _ = roc_curve(y_test, y_proba)
    plt.plot(fpr, tpr, label=f"NN-{mode} (AUC={auc:.3f})")
    plt.plot([0,1],[0,1],"--",color="gray")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("ROC Curve (Neural Net)")
    plt.legend()
    plt.show()

    # --- Predict unknowns ---
    unknown_proba = mlp.predict_proba(X_unknown_scaled)[:,1]
    df_unknown = df_scores.loc[y.isnull(), ['txId']].copy()
    df_unknown['prob_illicit'] = unknown_proba

    filename = f"suspicious_unknowns_nn_{mode}.csv"
    df_unknown.to_csv(filename, index=False)
    print(f"Saved top suspicious unknowns to {filename}")

    print("\nTop suspicious unknowns:")
    display(df_unknown.sort_values(by="prob_illicit", ascending=False).head(top_n))

    return df_unknown, auc

In [ ]:
for mode in ["raw", "cap", "drop"]:
    run_neural_net(df_scores, mode=mode, top_n=20)

## Simple Ensemble (Equal Weighting)

To build a baseline ensemble, we took the average of three model predictions: Logistic Regression (LR), Random Forest (RF), and Neural Network (NN). Formally:

$$ \text{Ensemble_Prob} = \frac{1}{3} (LR + RF + NN) $$

This approach treats each model equally, smoothing out individual weaknesses and producing a more stable probability score.

In [ ]:
# change to ['raw', 'cap', 'drop']
MODES = ["cap", "drop"]
MODELS = {
    "log": "suspicious_unknowns_log_{}.csv",
    "rf":  "suspicious_unknowns_rf_{}.csv",
    "nn":  "suspicious_unknowns_nn_{}.csv",
}

def load_model_df(model_key, pattern, modes=MODES):
    """
    Loads cap/drop for one model and returns:
      txId, prob_cap, prob_drop  (only the modes that exist)
    """
    parts = []
    for m in modes:
        f = Path(pattern.format(m))
        if f.exists():
            df = pd.read_csv(f)
            if "txId" not in df.columns or "prob_illicit" not in df.columns:
                raise ValueError(f"{f} must have columns: txId, prob_illicit")
            df = df[["txId", "prob_illicit"]].rename(columns={"prob_illicit": f"prob_{m}"})
            parts.append(df)
        else:
            print(f"[warn] Missing file for {model_key}-{m}: {f}")
    if not parts:
        return None
    # Outer-merge in case sets of unknown txIds differ by mode
    out = reduce(lambda l, r: pd.merge(l, r, on="txId", how="outer"), parts)
    return out

# Load each model's cap/drop files and compute per-model averages
model_avgs = []
for key, patt in MODELS.items():
    df_m = load_model_df(key, patt, modes=MODES)
    if df_m is None:
        continue
    # mean across available mode columns
    mode_cols = [c for c in df_m.columns if c.startswith("prob_")]
    df_m[f"{key}_avg"] = df_m[mode_cols].mean(axis=1, skipna=True)
    model_avgs.append(df_m[["txId", f"{key}_avg"]])

# Safety check
if not model_avgs:
    raise RuntimeError("No model files found. Make sure you saved your CSVs with the expected names.")

# Merge per-model averages
ensemble = reduce(lambda l, r: pd.merge(l, r, on="txId", how="outer"), model_avgs)

# Final ensemble across models (mean of available per-model avgs)
avg_cols = [c for c in ensemble.columns if c.endswith("_avg")]
ensemble["ensemble_prob"] = ensemble[avg_cols].mean(axis=1, skipna=True)

# Rank & save
ensemble = ensemble.sort_values("ensemble_prob", ascending=False).reset_index(drop=True)
ensemble.to_csv("suspicious_ensemble.csv", index=False)
print("Saved ensemble to suspicious_ensemble.csv")
print(ensemble.head(50))

This “flat-line” ensemble ensures no single model dominates. By averaging across LR, RF, and NN, we create a balanced risk score (ensemble_prob) that reduces noise and increases reliability in identifying suspicious transactions.

At the top of 50 risk scores.

## Ensemble of Three Models Using Probabilities

We combined three different model types—Logistic Regression, Random Forest, and Neural Network—into a single ensemble. A simple average of their probabilities provides a balanced score, since each model contributes equally.

However, in practice some models may perform better than others. In that case, it can be even more effective to create an **optimized ensemble**, where models are weighted differently (or stacked) to maximize overall performance.

In [ ]:
# You’ll need small wrappers that, given train_idx/test_idx and a mode ("cap"/"drop"),
# fit your model (LR/RF/NN) on X_train_known and output probabilities on X_valid_known and X_unknown.
# You already have run_logistic/run_random_forest/run_neural_net — adapt them to accept indices.

def get_base_oof_and_unknown(df_scores, build_and_predict_fn_list, modes=("cap","drop"), n_splits=5, random_state=42):
    """
    Returns:
      oof_df: DataFrame with columns ['txId','y_true', features...] for known examples
      unk_df: DataFrame with columns ['txId', features...] for unknown examples
    features include one column per (model,mode), e.g., 'log_cap', 'rf_drop', ...
    """
    X = df_scores.drop(columns=['txId','label'])
    y = df_scores['label'].map({'licit':0,'illicit':1})
    tx = df_scores['txId']

    known_mask = y.notnull()
    unk_mask = y.isnull()

    Xk, yk, txk = X[known_mask].reset_index(drop=True), y[known_mask].reset_index(drop=True), tx[known_mask].reset_index(drop=True)
    Xu, txu = X[unk_mask].reset_index(drop=True), tx[unk_mask].reset_index(drop=True)

    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)

    # Initialize storage
    oof = pd.DataFrame({"txId": txk, "y_true": yk})
    unk = pd.DataFrame({"txId": txu})

    # For each base model fn, and each mode, collect OOF predictions and fold-avg unknown preds
    for name, fn in build_and_predict_fn_list:  # e.g., [("log", log_fn), ("rf", rf_fn), ("nn", nn_fn)]
        for mode in modes:
            oof_col = f"{name}_{mode}"
            oof[oof_col] = np.nan
            unk_preds_folds = []

            for train_idx, valid_idx in skf.split(Xk, yk):
                # fn should: fit on (Xk.iloc[train_idx], yk.iloc[train_idx]) with mode,
                # return (valid_pred_proba_on_Xk_valid, unknown_pred_proba_on_Xu)
                valid_pred, unk_pred = fn(Xk.iloc[train_idx], yk.iloc[train_idx],
                                          Xk.iloc[valid_idx], Xu, mode)

                oof.loc[valid_idx, oof_col] = valid_pred
                unk_preds_folds.append(unk_pred)

            # unknown preds: average over folds
            unk[oof_col] = np.vstack(unk_preds_folds).mean(axis=0)

    return oof, unk


def logistic_wrapper(Xtr, ytr, Xval, Xunk, mode):
    # Apply your mode transform to shortest_path_to_illicit on Xtr/Xval/Xunk consistently.
    def transform_shortest_path(df):
        if "shortest_path_to_illicit" in df.columns:
            df = df.copy()
            if mode == "cap":
                df["shortest_path_to_illicit"] = df["shortest_path_to_illicit"].clip(upper=5)
            elif mode == "drop":
                df = df.drop(columns=["shortest_path_to_illicit"])
        return df

    Xtr_t = transform_shortest_path(Xtr)
    Xval_t = transform_shortest_path(Xval)
    Xunk_t = transform_shortest_path(Xunk)

    scaler = StandardScaler()
    Xtr_s = scaler.fit_transform(Xtr_t)
    Xval_s = scaler.transform(Xval_t)
    Xunk_s = scaler.transform(Xunk_t)

    clf = LogisticRegression(max_iter=1000, solver="liblinear")
    clf.fit(Xtr_s, ytr)

    val_pred = clf.predict_proba(Xval_s)[:,1]
    unk_pred = clf.predict_proba(Xunk_s)[:,1]
    return val_pred, unk_pred

# Build the list of base wrappers
build_and_predict_fn_list = [
    ("log", logistic_wrapper),
    # ("rf",   rf_wrapper),     # implement like logistic_wrapper but without scaling
    # ("nn",   nn_wrapper),     # implement with StandardScaler + MLPClassifier
]

# 1) Get OOF predictions (known) + averaged unknown predictions per base feature
oof_df, unk_df = get_base_oof_and_unknown(df_scores, build_and_predict_fn_list, modes=("cap","drop"), n_splits=5)

# 2) Train meta-model on OOF (known)
#    Choose features (all base-mode columns) and target y_true
meta_feats = [c for c in oof_df.columns if c not in ("txId","y_true")]
meta_clf = LogisticRegression(max_iter=1000)
meta_clf.fit(oof_df[meta_feats], oof_df["y_true"])

# 3) Apply meta-model to unknowns using the averaged base predictions
unk_df["ensemble_prob_stack"] = meta_clf.predict_proba(unk_df[meta_feats])[:,1]
ranked_unknowns = unk_df.sort_values("ensemble_prob_stack", ascending=False)

In [ ]:
print(ranked_unknowns.head(50))

Here is an optimize for the three models of assemble and here is top 50 risk score.

Next we'll do the percentage to see what the linear regression learning from the weights.

In [ ]:
# Use the exact matrix you trained on to guarantee alignment
X_fit = oof_df[meta_feats].copy()
feat_names = list(X_fit.columns)

# Coefficients (binary logistic => shape (1, n_features))
coefs = np.ravel(meta_clf.coef_)
intercept = float(np.ravel(meta_clf.intercept_)[0])

# Sanity check: lengths must match
if len(feat_names) != len(coefs):
    print(f"[WARN] Feature/coef length mismatch: feats={len(feat_names)}, coefs={len(coefs)}")
    # Fallback: align to the smaller length so you can still inspect
    L = min(len(feat_names), len(coefs))
    feat_names = feat_names[:L]
    coefs = coefs[:L]

coef_df = pd.DataFrame({"feature": feat_names, "coef": coefs})
# Split "log_cap" -> model="log", mode="cap"
parts = coef_df["feature"].str.split("_", n=1, expand=True)
coef_df["model"] = parts[0]
coef_df["mode"]  = parts[1]

print("=== Meta-model (stacker) coefficients ===")
print(coef_df.sort_values("coef", ascending=False).to_string(index=False))
print(f"\nIntercept: {intercept:+.6f}\n")

# Signed influence by base model
signed_sum = coef_df.groupby("model")["coef"].sum().sort_values(ascending=False)

# Positive-contribution percentages (stakeholder-friendly)
pos_df = coef_df.assign(pos_coef=np.clip(coef_df["coef"], 0, None))
pos_sum = pos_df.groupby("model")["pos_coef"].sum()

# If everything is <= 0, fall back to absolute values
if pos_sum.sum() == 0:
    print("Note: all learned coefficients are <= 0; using absolute values for percentages as a fallback.\n")
    pos_sum = coef_df.groupby("model")["coef"].apply(np.abs).sum()

percent = (pos_sum / pos_sum.sum() * 100).sort_values(ascending=False)

print("=== Model-level influence (signed sum of coefs) ===")
for m, v in signed_sum.items():
    print(f"{m:>3}: {v:+.6f}")

print("\n=== Model-level contribution (percent of positive weight) ===")
for m, v in percent.items():
    print(f"{m:>3}: {v:5.1f}%")

# Optional: see whether 'cap' vs 'drop' matters per model
per_mode = pos_df.groupby(["model","mode"])["pos_coef"].sum()
print("\n=== Positive contribution by model × mode ===")
print(per_mode.to_string())

Interpreting the Stacking Weights (cap vs drop)

In our stacking setup, we trained two variants of the logistic model:

cap: uses a capped version of shortest_path_to_illicit (e.g., clip at 5).

drop: excludes shortest_path_to_illicit.

The stacker learned a much larger weight for the cap variant than for drop, which means the capped shortest-path signal is highly predictive. This matches our earlier finding that shortest path to an illicit node is one of the strongest features across models.

Important: The “percentages” we printed right now reflect only the logistic model’s two variants (cap vs drop), because RF and NN aren’t included yet. So it’s correct to say “cap contributes more than drop,” but it’s not yet a three-model split (LR vs RF vs NN).

Next step to get true ensemble percentages: add the RF and NN wrappers to the stack, re-fit, and then read the model-level percentages (LR / RF / NN). Those learned weights are what you’d use to describe or implement an optimized three-model ensemble.

### OOF Stacked Meta-Model

We extend beyond a single model by stacking the outputs of Logistic Regression, Random Forest, and Neural Network. Each base model produces out-of-fold (OOF) predictions, and then a meta-model (logistic regression) learns how to combine them into a single probability score.

This approach allows us to use the strengths of all three models together, rather than relying on one alone. The meta-model effectively decides how much weight each base model should receive, creating a more optimized ensemble probability.

In [ ]:
# --- helper: apply the mode transform consistently ---
def transform_shortest_path(df, mode):
    if "shortest_path_to_illicit" in df.columns:
        df = df.copy()
        if mode == "cap":
            df["shortest_path_to_illicit"] = df["shortest_path_to_illicit"].clip(upper=5)
        elif mode == "drop":
            df = df.drop(columns=["shortest_path_to_illicit"])
    return df

# --- base model wrappers: each returns (val_pred, unk_pred) ---

def logistic_wrapper(Xtr, ytr, Xval, Xunk, mode):
    Xtr_t = transform_shortest_path(Xtr, mode)
    Xval_t = transform_shortest_path(Xval, mode)
    Xunk_t = transform_shortest_path(Xunk, mode)

    scaler = StandardScaler()
    Xtr_s = scaler.fit_transform(Xtr_t)
    Xval_s = scaler.transform(Xval_t)
    Xunk_s = scaler.transform(Xunk_t)

    clf = LogisticRegression(max_iter=1000, solver="liblinear")
    clf.fit(Xtr_s, ytr)
    return clf.predict_proba(Xval_s)[:,1], clf.predict_proba(Xunk_s)[:,1]

def rf_wrapper(Xtr, ytr, Xval, Xunk, mode):
    Xtr_t = transform_shortest_path(Xtr, mode)
    Xval_t = transform_shortest_path(Xval, mode)
    Xunk_t = transform_shortest_path(Xunk, mode)

    rf = RandomForestClassifier(
        n_estimators=300, max_depth=None,
        class_weight="balanced", n_jobs=-1, random_state=42
    )
    rf.fit(Xtr_t, ytr)
    return rf.predict_proba(Xval_t)[:,1], rf.predict_proba(Xunk_t)[:,1]

def nn_wrapper(Xtr, ytr, Xval, Xunk, mode):
    Xtr_t = transform_shortest_path(Xtr, mode)
    Xval_t = transform_shortest_path(Xval, mode)
    Xunk_t = transform_shortest_path(Xunk, mode)

    scaler = StandardScaler()
    Xtr_s = scaler.fit_transform(Xtr_t)
    Xval_s = scaler.transform(Xval_t)
    Xunk_s = scaler.transform(Xunk_t)

    mlp = MLPClassifier(hidden_layer_sizes=(64,32), activation='relu',
                        solver='adam', alpha=1e-4, max_iter=200, random_state=42)
    mlp.fit(Xtr_s, ytr)
    return mlp.predict_proba(Xval_s)[:,1], mlp.predict_proba(Xunk_s)[:,1]

# --- stacking utilities you already have (kept for completeness) ---

def get_base_oof_and_unknown(df_scores, build_and_predict_fn_list, modes=("cap","drop"), n_splits=5, random_state=42):
    X = df_scores.drop(columns=['txId','label'])
    y = df_scores['label'].map({'licit':0,'illicit':1})
    tx = df_scores['txId']

    known_mask = y.notnull()
    unk_mask = y.isnull()

    Xk, yk, txk = X[known_mask].reset_index(drop=True), y[known_mask].reset_index(drop=True), tx[known_mask].reset_index(drop=True)
    Xu, txu = X[unk_mask].reset_index(drop=True), tx[unk_mask].reset_index(drop=True)

    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)

    oof = pd.DataFrame({"txId": txk, "y_true": yk})
    unk = pd.DataFrame({"txId": txu})

    for name, fn in build_and_predict_fn_list:
        for mode in modes:
            col = f"{name}_{mode}"
            oof[col] = np.nan
            unk_fold_preds = []
            for tr_idx, va_idx in skf.split(Xk, yk):
                val_pred, unk_pred = fn(Xk.iloc[tr_idx], yk.iloc[tr_idx],
                                        Xk.iloc[va_idx], Xu, mode)
                oof.loc[va_idx, col] = val_pred
                unk_fold_preds.append(unk_pred)
            unk[col] = np.vstack(unk_fold_preds).mean(axis=0)
    return oof, unk

In [ ]:
# 1) Collect OOF preds for knowns + averaged preds for unknowns
build_and_predict_fn_list = [
    ("log", logistic_wrapper),
    ("rf",  rf_wrapper),
    ("nn",  nn_wrapper),
]
oof_df, unk_df = get_base_oof_and_unknown(df_scores, build_and_predict_fn_list, modes=("cap","drop"), n_splits=5)

# 2) Train a simple meta-model on OOF predictions (to avoid leakage)
meta_feats = [c for c in oof_df.columns if c not in ("txId","y_true")]
meta = LogisticRegression(max_iter=1000, solver="liblinear")
meta.fit(oof_df[meta_feats], oof_df["y_true"])

# 3) Apply meta-model to unknowns (using fold-averaged base preds)
unk_df["ensemble_prob_stack"] = meta.predict_proba(unk_df[meta_feats])[:,1]
ranked_unknowns = unk_df.sort_values("ensemble_prob_stack", ascending=False)

# Save & preview
ranked_unknowns.to_csv("suspicious_unknowns_stacked.csv", index=False)
print(ranked_unknowns.head(20))

Here's the top of all three models and let's go see how much probability from those models are.

In [ ]:
# Use exactly what the meta-model saw to avoid any mismatch
X_fit = oof_df[meta_feats].copy()
feat_names = list(X_fit.columns)

# Coefs & intercept (binary logistic: shape (1, n_features))
coefs = np.ravel(meta.coef_)
intercept = float(np.ravel(meta.intercept_)[0])

# Safety check
if len(feat_names) != len(coefs):
    print(f"[WARN] Feature/coef length mismatch: feats={len(feat_names)}, coefs={len(coefs)}")
    L = min(len(feat_names), len(coefs))
    feat_names, coefs = feat_names[:L], coefs[:L]

# Build coef table
coef_df = pd.DataFrame({"feature": feat_names, "coef": coefs})
parts = coef_df["feature"].str.split("_", n=1, expand=True)
coef_df["model"] = parts[0]   # 'log' | 'rf' | 'nn'
coef_df["mode"]  = parts[1]   # 'cap' | 'drop'

print("\n=== Stacker coefficients per feature ===")
print(coef_df.sort_values("coef", ascending=False).to_string(index=False))
print(f"\nIntercept: {intercept:+.6f}\n")

# 1) Signed influence by model (positive pushes risk up)
signed_sum = coef_df.groupby("model")["coef"].sum().sort_values(ascending=False)

# 2) Stakeholder-friendly % split:
#    Use only positive mass (negatives clipped to 0) so we don’t give % credit to down-weights.
pos_df = coef_df.assign(pos_coef=np.clip(coef_df["coef"], 0, None))
pos_sum = pos_df.groupby("model")["pos_coef"].sum()

# Fallback: if all coefs <= 0, use absolute values to show relative magnitude
if pos_sum.sum() == 0:
    print("Note: all learned coefficients are <= 0; using absolute magnitudes for percentages.\n")
    pos_sum = coef_df.groupby("model")["coef"].apply(np.abs).sum()

percent = (pos_sum / pos_sum.sum() * 100).sort_values(ascending=False)

print("=== Model-level influence (signed sum of coefs) ===")
for m, v in signed_sum.items():
    print(f"{m:>3}: {v:+.6f}")

print("\n=== Model-level contribution (percent of positive weight) ===")
for m, v in percent.items():
    print(f"{m:>3}: {v:5.1f}%")

# Optional: see cap vs drop within each model
per_mode = pos_df.groupby(["model","mode"])["pos_coef"].sum()
print("\n=== Positive contribution by model × mode ===")
print(per_mode.to_string())

So here's the contribution of the percentage of the weights: RF: 43.2%, LR: 32.4%, NN: 24.4%. Now we're using a average and the weight and we'll merge those in. And now we'll try to sort all of it using the best both worlds for the average and the waited to find unknown nodes of high probability of illicit.

Next using average and a weighted score.

In [ ]:
# --- 1) Build per-model averages from your cap/drop columns ---
need = ["log_cap","log_drop","rf_cap","rf_drop","nn_cap","nn_drop"]
missing = [c for c in need if c not in unk_df.columns]
if missing:
    raise KeyError(f"Expected base columns missing from unk_df: {missing}")

unk_df = unk_df.copy()
unk_df["log_avg"] = unk_df[["log_cap","log_drop"]].mean(axis=1)
unk_df["rf_avg"]  = unk_df[["rf_cap","rf_drop"]].mean(axis=1)
unk_df["nn_avg"]  = unk_df[["nn_cap","nn_drop"]].mean(axis=1)

# Simple (non-stacked) ensemble as mean of available model avgs
unk_df["ensemble_prob"] = unk_df[["log_avg","rf_avg","nn_avg"]].mean(axis=1)

# --- 2) Build merge inputs ---
ens   = unk_df[["txId","log_avg","rf_avg","nn_avg","ensemble_prob"]].copy()
stack = ranked_unknowns[["txId","ensemble_prob_stack"]].copy()

ens["txId"]   = ens["txId"].astype(str)
stack["txId"] = stack["txId"].astype(str)

# --- 3) Merge + rank + percentiles + save ---
final = ens.merge(stack, on="txId", how="outer")

final["rank"] = final["ensemble_prob"].rank(ascending=False, method="first").astype(int)
final["percentile"] = (1 - final["rank"] / len(final)) * 100

final = final.sort_values("ensemble_prob", ascending=False)[
    ["txId","log_avg","rf_avg","nn_avg","ensemble_prob","ensemble_prob_stack","rank","percentile"]
]

final.to_csv("suspicious_unknowns_final.csv", index=False)
print(final.head(20))

# --- 4) Top-3 for your Stakeholder Summary table ---
top3 = final.head(3).copy()
top3["Percentile Rank"] = top3["percentile"].round(2).astype(str) + "%"
top3_table = top3[["txId","ensemble_prob","Percentile Rank"]].rename(
    columns={"ensemble_prob": "Ensemble Probability (Illicit)"}
)
print("\nTop 3 suspicious unknowns:")
print(top3_table.to_string(index=False))

In [ ]:
# Expect: unk_df has txId, log_cap, log_drop, rf_cap, rf_drop, nn_cap, nn_drop
#         ranked_unknowns has txId, ensemble_prob_stack

# 0) Validate columns
need = ["txId","log_cap","log_drop","rf_cap","rf_drop","nn_cap","nn_drop"]
missing = [c for c in need if c not in unk_df.columns]
if missing:
    raise KeyError(f"Expected base columns missing from unk_df: {missing}")

unk_df = unk_df.copy()
unk_df["txId"] = unk_df["txId"].astype(str)

# 1) Per-model averages (cap/drop → avg)
unk_df["log_avg"] = unk_df[["log_cap","log_drop"]].mean(axis=1)
unk_df["rf_avg"]  = unk_df[["rf_cap","rf_drop"]].mean(axis=1)
unk_df["nn_avg"]  = unk_df[["nn_cap","nn_drop"]].mean(axis=1)

# 2) Equal-weight ensemble across the 3 model avgs
unk_df["ensemble_prob_equal"] = unk_df[["log_avg","rf_avg","nn_avg"]].mean(axis=1)

# 3) Optimized weighted ensemble using learned percentages (from stacker)
w_log, w_rf, w_nn = 0.324, 0.432, 0.244  # LR 32.4%, RF 43.2%, NN 24.4%
unk_df["ensemble_prob_weighted"] = (
    w_log*unk_df["log_avg"] + w_rf*unk_df["rf_avg"] + w_nn*unk_df["nn_avg"]
)

# 4) Bring in stacked score
stack = ranked_unknowns[["txId","ensemble_prob_stack"]].copy()
stack["txId"] = stack["txId"].astype(str)

final = (
    unk_df[["txId","log_avg","rf_avg","nn_avg",
            "ensemble_prob_equal","ensemble_prob_weighted"]]
    .merge(stack, on="txId", how="inner")
)

# 5) Rank each score separately + percentiles
for col in ["ensemble_prob_equal", "ensemble_prob_weighted", "ensemble_prob_stack"]:
    final[f"rank_{col}"] = final[col].rank(ascending=False, method="first").astype(int)
n = len(final)
for col in ["ensemble_prob_equal", "ensemble_prob_weighted", "ensemble_prob_stack"]:
    final[f"pct_{col}"] = (1 - final[f"rank_{col}"] / n) * 100

# 6) Consensus rank (average of ranks; lower = riskier) + consensus percentile
final["rank_consensus"] = final[
    ["rank_ensemble_prob_equal","rank_ensemble_prob_weighted","rank_ensemble_prob_stack"]
].mean(axis=1)
final = final.sort_values("rank_consensus", ascending=True).reset_index(drop=True)
final["percentile_consensus"] = (1 - (final["rank_consensus"] - 1) / len(final)) * 100

# 7) High-agreement flag (top 5% in both equal + stacked)
final["high_agreement"] = (
    (final["pct_ensemble_prob_equal"] >= 95.0) &
    (final["pct_ensemble_prob_stack"] >= 95.0)
)

# 8) Save + preview
cols_out = [
    "txId","log_avg","rf_avg","nn_avg",
    "ensemble_prob_equal","ensemble_prob_weighted","ensemble_prob_stack",
    "rank_ensemble_prob_equal","rank_ensemble_prob_weighted","rank_ensemble_prob_stack",
    "percentile_consensus","high_agreement"
]
final[cols_out].to_csv("suspicious_unknowns_final.csv", index=False)
print("Saved -> suspicious_unknowns_final.csv")
print(final[cols_out].head(20))

# 9) Tiny stakeholder table
top3 = final.head(3).copy()
top3_tbl = top3[["txId","ensemble_prob_weighted","ensemble_prob_stack",
                 "percentile_consensus","high_agreement"]].rename(columns={
    "ensemble_prob_weighted": "Weighted Risk",
    "ensemble_prob_stack": "Stacked Risk",
    "percentile_consensus": "Consensus Percentile"
})
print("\nTop 3 suspicious unknowns (consensus):")
print(top3_tbl.to_string(index=False))

**Consensus percentile** is your way of saying: “This node is ranked highly by both the equal-weight ensemble and the weighted/stacked probabilities.” It averages the rankings from equal-weight, weighted (learned percentages), and stacked meta-model.

Sanity check with models score

In [ ]:
# OOF has y_true and the same meta_feats (log_cap/drop, rf_cap/drop, nn_cap/drop)
oof_preds_avg   = oof_df[["log_cap","log_drop","rf_cap","rf_drop","nn_cap","nn_drop"]].mean(axis=1)
oof_preds_stack = meta.predict_proba(oof_df[meta_feats])[:,1]  # same meta as you trained

def summarize(name, y, p):
    auc = roc_auc_score(y, p)
    brier = brier_score_loss(y, p)
    print(f"{name:22s}  AUC={auc:.3f}  Brier={brier:.4f}")

y = oof_df["y_true"].values
summarize("avg ensemble (OOF)", y, oof_preds_avg.values)
summarize("stacked (OOF)",      y, oof_preds_stack)

**Key points:**
- Both the average and stacked ensembles achieved **perfect separation (AUC=1.0)** on labeled validation data.  
- The stacked meta-model gave **sharper calibration** (lower Brier score).  
- For stakeholders, we report the **average ensemble (`ensemble_prob`)** because its values are easier to interpret (e.g., 0.64 ≈ 64% chance illicit).  
- The stacked scores (`ensemble_prob_stack`) are **retained** in the dataset for analysts who want to experiment with alternative ranking strategies.  

## Results

### Models Tested

We built and evaluated three different models:

Logistic regression for interpretability

Random forest for tree-based pattern recognition

Neural network for capturing non-linear structure

### Ensemble Approach

Rather than relying on a single model, we combined them into an ensemble. This approach balances recall (catching more suspicious cases) with precision (minimizing false alarms).

The ensemble improves over individual models by capturing complementary signals. Transactions most likely to be illicit show consistent agreement across methods.

### Stakeholder Illustration

The ensemble flagged several unknown transactions as high-probability illicit. For illustration, here are three of the top cases identified.

In [ ]:


# =========================
# Helpers
# =========================
def _bfs_hops(G, center, radius=4):
    """Return dict: node -> hop distance (0..radius) from center in G."""
    center = str(center)
    if center not in G:
        return {center: 0}
    dist = {center: 0}
    q = deque([center])
    while q:
        v = q.popleft()
        if dist[v] == radius:
            continue
        for w in G[v]:
            if w not in dist:
                dist[w] = dist[v] + 1
                q.append(w)
    return dist

def _radial_positions_by_hop(hops, jitter=0.03):
    """Place nodes on concentric rings by hop distance around (0.5,0.5)."""
    by_hop = {}
    for n, d in hops.items():
        by_hop.setdefault(d, []).append(n)
    pos = {}
    R = max(by_hop) if by_hop else 0
    for d, nodes in by_hop.items():
        r = 0.18 + 0.74 * (d / max(1, R))  # normalized radius
        k = len(nodes)
        for i, n in enumerate(nodes):
            angle = 2 * math.pi * (i / max(1, k))
            x = 0.5 + r * math.cos(angle) + np.random.uniform(-jitter, jitter)
            y = 0.5 + r * math.sin(angle) + np.random.uniform(-jitter, jitter)
            pos[n] = (x, y)
    return pos

def _node_color(n, class_map):
    c = class_map.get(str(n), "unknown")
    if c == "illicit": return "#d62728"   # red
    if c == "licit":   return "#2ca02c"   # green
    return "#7f7f7f"                      # gray

def _focused_subgraph(G, tx, class_map, illicit_set, radius=4, connect_all_illicit=True):
    """
    Keep a readable subset around tx:
      - always center + 1-hop neighbors
      - all illicit nodes within 'radius'
      - shortest path(s) from center to illicit:
          * if connect_all_illicit=True => to every illicit within radius
          * else only to the nearest illicit
    """
    tx = str(tx)
    if tx not in G:
        H = nx.Graph(); H.add_node(tx); return H

    keep = set(nx.ego_graph(G, tx, radius=1).nodes())
    ego_big = nx.ego_graph(G, tx, radius=radius)
    illicit_in_ego = {n for n in ego_big if n in illicit_set}

    lengths = nx.single_source_shortest_path_length(G, tx, cutoff=radius)

    if illicit_in_ego:
        if connect_all_illicit:
            for target in illicit_in_ego:
                if target in lengths:
                    keep |= set(nx.shortest_path(G, tx, target))
        else:
            target = min(illicit_in_ego, key=lambda n: lengths.get(n, float("inf")))
            if target in lengths:
                keep |= set(nx.shortest_path(G, tx, target))

    keep |= illicit_in_ego  # keep the illicit nodes themselves
    H = G.subgraph(keep).copy()

    # Ensure string labels
    if len(H) and not isinstance(next(iter(H.nodes())), str):
        H = nx.relabel_nodes(H, lambda n: str(n), copy=True)
    return H

# =========================
# Plotter
# =========================
def plot_candidate_brief(
    tx,
    final,
    G_undirected,
    class_map,
    SHOW_COL="ensemble_prob_weighted",      # score to DISPLAY (prob or percentile)
    PRED_PROB_COL="ensemble_prob_weighted", # prob (0..1) used for neighbor flags
    p_thresh=0.50,
    radius=4,
    focus=True,
    connect_all_illicit=True,               # if focus=True, connect to ALL illicit nodes
    figsize=(12, 5.5),
    save_path=None,
):
    """
    SHOW_COL can be: 'ensemble_prob_equal', 'ensemble_prob_weighted',
                     'ensemble_prob_stack', 'percentile_consensus'
    PRED_PROB_COL must be a probability column in [0,1] (recommended: weighted).
    """

    tx = str(tx)
    assert SHOW_COL in final.columns, f"{SHOW_COL} not found in final.columns"
    assert PRED_PROB_COL in final.columns, f"{PRED_PROB_COL} not found in final.columns"

    # Maps
    final = final.copy()
    final["txId"] = final["txId"].astype(str)
    prob_map_show = dict(zip(final["txId"], final[SHOW_COL].values))
    prob_map_pred = dict(zip(final["txId"], final[PRED_PROB_COL].values))

    show_val = float(prob_map_show.get(tx, float("nan")))
    title_val = f"{show_val:.2f}" if "percentile" in SHOW_COL.lower() else f"{show_val:.3f}"

    illicit_set = {str(k) for k, v in class_map.items() if v == "illicit"}

    # Subgraph we will actually draw + hop distances on THIS subgraph
    if focus:
        H = _focused_subgraph(G_undirected, tx, class_map, illicit_set,
                              radius=max(radius, 2), connect_all_illicit=connect_all_illicit)
    else:
        H = nx.ego_graph(G_undirected, tx, radius=radius)
        if len(H) and not isinstance(next(iter(H.nodes())), str):
            H = nx.relabel_nodes(H, lambda n: str(n), copy=True)

    hops = nx.single_source_shortest_path_length(H, tx, cutoff=radius)

    # Layout
    pos = _radial_positions_by_hop(hops)
    # Backfill any missing positions
    if len(pos) < H.number_of_nodes():
        for n in H.nodes():
            if n not in pos:
                pos[n] = (0.5 + np.random.uniform(-0.02, 0.02),
                          0.5 + np.random.uniform(-0.02, 0.02))

    # Neighbor metrics (1-hop in the full graph)
    nbrs = list(G_undirected.neighbors(tx)) if tx in G_undirected else []
    labels = [class_map.get(str(n), "unknown") for n in nbrs]
    deg = len(nbrs)
    illicit_n = sum(z == "illicit" for z in labels)
    licit_n   = sum(z == "licit"   for z in labels)
    unknown_n = sum(z == "unknown" for z in labels)
    pred_illicit_unknown = sum(
        1 for n in nbrs
        if class_map.get(str(n), "unknown") == "unknown"
        and prob_map_pred.get(str(n), 0.0) >= p_thresh
    )

    # Shortest path to any illicit (bounded in full graph)
    sp = math.inf
    if tx in G_undirected and illicit_set:
        dist_full = _bfs_hops(G_undirected, tx, radius=radius)
        candidates = [n for n in dist_full if n in illicit_set and n != tx]
        if candidates:
            sp = min(dist_full[n] for n in candidates)
    sp_txt = "∞" if math.isinf(sp) else str(sp)

    # ----- Draw -----
    fig = plt.figure(figsize=figsize)
    gs = fig.add_gridspec(1, 2, width_ratios=[2.1, 1.0])
    axG = fig.add_subplot(gs[0, 0]); axB = fig.add_subplot(gs[0, 1])

    nx.draw_networkx_edges(H, pos, ax=axG, alpha=0.28, width=0.9)
    nx.draw_networkx_nodes(
        H, pos,
        nodelist=list(H.nodes()),
        node_color=[_node_color(n, class_map) for n in H.nodes()],
        node_size=90, ax=axG
    )
    if tx in H:
        # center node (bigger + black outline)
        nx.draw_networkx_nodes(
            H, pos, nodelist=[tx],
            node_color=[_node_color(tx, class_map)],
            edgecolors="black", linewidths=1.6, node_size=280, ax=axG
        )
        # halo
        circ = plt.Circle(pos[tx], radius=0.06, color="black", fill=False, lw=1.6, alpha=0.6)
        axG.add_patch(circ)

    # hop rings (0..R)
    if hops:
        R = max(hops.values())
        for d in range(0, R + 1):
            r = 0.18 + 0.74 * (d / max(1, R))
            ring = plt.Circle((0.5, 0.5), r, color="lightgray", fill=False, lw=0.6, alpha=0.35)
            axG.add_patch(ring)
            axG.text(0.5, 0.5 + r, f"hop {d}", ha="center", va="bottom", fontsize=8, alpha=0.6)

    axG.legend(handles=[
        Patch(facecolor="#d62728", edgecolor="none", label="illicit (known)"),
        Patch(facecolor="#2ca02c", edgecolor="none", label="licit (known)"),
        Patch(facecolor="#7f7f7f", edgecolor="none", label="unknown (unlabeled)"),
    ], loc="upper right", fontsize=8, frameon=False)

    axG.set_axis_off(); axG.margins(0.08)

    # Right metrics panel
    axB.axis("off")
    ring_counts = Counter(hops.values())
    ring_line = " | ".join(f"{k}-hop:{v}" for k, v in sorted(ring_counts.items()))
    lines = [
        f"{SHOW_COL}:                {title_val}",
        f"Degree (1-hop):           {deg}",
        f"Illicit neighbors:        {illicit_n}/{deg} ({(illicit_n/deg if deg else 0):.2f})",
        f"Pred illicit (unknown):   {pred_illicit_unknown}/{unknown_n} (p ≥ {p_thresh:.2f})",
        f"Shortest path to illicit: {sp_txt}",
        f"Hop counts:               {ring_line}",
    ]
    y = 0.95
    for s in lines:
        axB.text(0.02, y, s, transform=axB.transAxes, ha="left", va="top", fontsize=10)
        y -= 0.12

    fig.suptitle(f"txId: {tx}  |  {SHOW_COL}={title_val}  |  deg={deg}  |  sp_illicit={sp_txt}",
                 fontsize=11, y=1.02)
    fig.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches="tight"); plt.close()
    else:
        plt.show()

    return {
        "txId": tx, "show_col": SHOW_COL, "show_val": show_val,
        "pred_prob_col": PRED_PROB_COL, "p_thresh": p_thresh,
        "degree": deg, "illicit_neighbors": illicit_n, "licit_neighbors": licit_n,
        "unknown_neighbors": unknown_n, "pred_illicit_unknown": pred_illicit_unknown,
        "shortest_path_to_illicit": None if math.isinf(sp) else int(sp),
        "radius": int(radius), "saved": bool(save_path), "save_path": save_path,
    }

# =========================
# Usage examples
# =========================
# 1) Harmonize IDs (run once in your notebook before plotting)
final["txId"] = final["txId"].astype(str)
class_map = {str(k): v for k, v in class_map.items()}
if len(G_undirected) > 0 and not isinstance(next(iter(G_undirected.nodes())), str):
    G_undirected = nx.relabel_nodes(G_undirected, lambda n: str(n), copy=True)

# 2) Choose prob column for neighbor flags (0..1)
PRED_PROB_COL = "ensemble_prob_weighted"
THRESH = 0.50

# 3) Top-3 by equal-weight (full ego, with rings)
for tx in final.sort_values("ensemble_prob_equal", ascending=False).head(3)["txId"]:
    info = plot_candidate_brief(
        tx=tx,
        final=final,
        G_undirected=G_undirected,
        class_map=class_map,
        SHOW_COL="ensemble_prob_equal",
        PRED_PROB_COL=PRED_PROB_COL,
        p_thresh=THRESH,
        radius=4,
        focus=False,                # full ego neighborhood
        connect_all_illicit=True,   # (used only if focus=True)
        save_path=None,             # show inline
    )
    print(info)

# 4) Top-3 by optimized weighted (full ego, with rings)
for tx in final.sort_values("ensemble_prob_weighted", ascending=False).head(3)["txId"]:
    info = plot_candidate_brief(
        tx=tx,
        final=final,
        G_undirected=G_undirected,
        class_map=class_map,
        SHOW_COL="ensemble_prob_weighted",
        PRED_PROB_COL=PRED_PROB_COL,
        p_thresh=THRESH,
        radius=4,
        focus=False,                # full ego neighborhood
        connect_all_illicit=True,   # (used only if focus=True)
        save_path=None,
    )
    print(info)

Node `208930302` Has the **risk score = 0.641** Which we use the features and models and doing ensemble of `PROB_COL = "ensemble_prob"`. The **degree is 1** And the **shortest path is also 1** to a illicit. The Pred illicit for unknowns are zero, There are no unlabeled neighbors. The **illicit neighbors are all 1** meaning $\frac{\text{illicit}}{\text{illict + licit + unknown}} = [0,1]$ probability range.

In other words, this transaction sits in a **bad neighborhood**: its only direct neighbor is already labeled illicit, and there are no unlabeled or licit neighbors to offset this signal. The ensemble probability places this node in a high-risk range, consistent with proximity-based indicators.

### Wrap-Up

Well, that’s a wrap folks!

Here are the stakeholder visuals and results — the top suspicious unknowns, highlighted by multiple ensemble approaches. These nodes consistently show patterns that put them very close to being classified as illicit.

If you’re still reading, thank you for following along with my project and for your attention!

## References

[1] M. P. Deisenroth, A. A. Faisal, and C. S. Ong, Mathematics for Machine Learning. Cambridge, U.K.: Cambridge Univ. Press, 2020.

[2] V. H. Masías, M. Valle, C. Morselli, F. Crespo, A. Vargas, and S. Laengle, “Modeling verdict outcomes using social network measures: The Watergate and Caviar Network cases,” PLOS ONE, vol. 11, no. 1, p. e0147248, Jan. 2016.

[3] S. Masihullah, M. Negi, M. Joe, and J. Sathyanarayana, “Identifying fraud rings using domain-aware weighted community detection,” Medium, 2022. [Online]. Available: https://medium.com/@example/article-link

[4] N. Mukhopadhyay, Probability and Statistical Inference. New York, NY, USA: Marcel Dekker, 2000.

[5] M. E. J. Newman, Networks: An Introduction. Oxford, U.K.: Oxford Univ. Press, 2010.

[6] H. Schenck, Algebraic Foundations for Applied Topology and Data Analysis. Cham, Switzerland: Springer, 2022.

[7] L. N. Trefethen and D. Bau III, Numerical Linear Algebra. Philadelphia, PA, USA: SIAM, 1997.

[8] Elliptic. [Online]. Available: https://www.elliptic.co

[9] M. Weber, G. Domeniconi, J. Chen, D. K. I. Weidele, C. Bellei, T. Robinson, and C. E. Leiserson, “Anti-money laundering in Bitcoin: Experimenting with graph convolutional networks for financial forensics,” in Proc. KDD Workshop Anomaly Detection Finance, New York, NY, USA, Aug. 2019.

[10] D. B. West, Introduction to Graph Theory. Boston, MA, USA: Pearson Educ., 2001.

[11] G. Nikolentzos, G. Dasoulas, and M. Vazirgiannis, “k-hop graph neural networks,” in Proc. 38th Int. Conf. Machine Learning (ICML), 2021. [Online]. Available: https://arxiv.org/abs/2011.05303

[12] D. B. Johnson, “Finding all the elementary circuits of a directed graph,” SIAM J. Comput., vol. 4, no. 1, pp. 77–84, 1975.

[13] U. Brandes, “A Faster Algorithm for Betweenness Centrality,” Journal of Mathematical Sociology, vol. 25, no. 2, pp. 163–177, 2001. doi:10.1080/0022250X.2001.9990249.

[14] H. Edelsbrunner and J. L. Harer, Computational Topology: An Introduction. Providence, RI, USA: American Mathematical Society, 2010.

[15] L. Page, S. Brin, R. Motwani, and T. Winograd, "The PageRank Citation Ranking: Bringing Order to the Web. Technical Report," Stanford InfoLab, 1999.

[16] A. A. Hagberg, D. A. Schult, and P. J. Swart, “Exploring network structure, dynamics, and function using NetworkX,” in Proceedings of the 7th Python in Science Conference (SciPy2008), G. Varoquaux, T. Vaught, and J. Millman, Eds., Pasadena, CA, USA, Aug. 2008, pp. 11–15.

[17] K. Paton, "An algorithm for finding a fundamental set of cycles of a graph," Communications of the ACM, vol. 12, no. 9, pp. 514–518, 1969.

[18] V. D. Blondel, J. L. Guillaume, R. Lambiotte, and E. Lefebvre, "Fast unfolding of communities in large networks," Journal of Statistical Mechanics: Theory and Experiment, vol. 2008, no. 10, p. P10008, 2008.

[19] D. J. Watts and S. H. Strogatz, "Collective dynamics of 'small-world' networks," Nature, vol. 393, no. 6684, pp. 440–442, 1998.

[20] A. Grover and J. Leskovec, node2vec: Scalable Feature Learning for Networks, Proceedings of the 22nd ACM SIGKDD International Conference on Knowledge Discovery and Data Mining (KDD), pp. 855–864, Aug. 2016. doi: 10.1145/2939672.2939754
.

[21] M. E. J. Newman, "Spectral methods for community detection and graph partitioning," Philosophical Transactions of the Royal Society A: Mathematical, Physical and Engineering Sciences, vol. 373, no. 2056, p. 20150120, 2015.

[22] R. Ghrist, "Barcodes: The persistent topology of data," Bulletin of the American Mathematical Society, vol. 45, no. 1, pp. 61–75, 2008.

In [ ]:
# This strips all outputs (plots, printouts, widget junk). Leaves only code + markdown.
# GitHub can render it without errors.
!jupyter nbconvert --ClearOutputPreprocessor.enabled=True --inplace Elliptic.ipynb

In [ ]:
!jupyter nbconvert --ClearOutputPreprocessor.enabled=True \
  --to notebook --output Elliptic_clean.ipynb Elliptic.ipynb